<a href="https://colab.research.google.com/github/WangKun00293/bart-bert-project/blob/main/BART_generate.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install datasets
from datasets import load_dataset
ag_news = load_dataset("ag_news")
## load dataset
import pandas as pd
train=ag_news["train"].to_pandas()
test=ag_news["test"].to_pandas()
from sklearn.model_selection import train_test_split

train_2=train[train["label"]==2].sample(n=2000, random_state=13)
train_3=train[train["label"]==3].sample(n=2000, random_state=13)
df_train=pd.concat([train_2,train_3]).sample(frac=1, random_state=13).reset_index(drop=True)

test_2=test[test["label"]==2].sample(n=1000, random_state=13)
test_3=test[test["label"]==3].sample(n=1000, random_state=13)
test_set=pd.concat([test_2,test_3]).sample(frac=1, random_state=13).reset_index(drop=True)

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 547.8/547.8 kB 9.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 9.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 316.1/316.1 kB 24.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 39.9/39.9 MB 25.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 7.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 15.1 MB/s eta 0:00:00
  Attempting uninstall: pyarrow
    Found existing installation: pyarrow 14.0.2
    Uninstalling pyarrow-14.0.2:
      Successfully uninstalled pyarrow-14.0.2
  Attempting uninstall: fsspec
    Found existing installation: fsspec 2024.6.1
    Uninstalling fsspec-2024.6.1:
      Successfully uninstalled fsspec-2024.6.1
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
torch 2.3.1+cu121 requir

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


Generating train split:   0%|          | 0/120000 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/7600 [00:00<?, ? examples/s]

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import re
def clean_text(text):
    text = re.sub(r'http\S+|www.\S+', '', text)
    text = re.sub(r'[&\\@#\$%\^*()_+={}\[\]:;"\'<>\|`~]', '', text)

    return text

In [ ]:
df_train['text'] = df_train['text'].apply(clean_text)
test_set['text'] = test_set['text'].apply(clean_text)

df_train['label'] = df_train['label'].replace({2: 0, 3: 1})
test_set['label'] = test_set['label'].replace({2: 0, 3: 1})

In [ ]:
label_map = {0: "Write a Business News", 1: "Write a Science/Technology News"}
# label_map2 = {"Write a Business News":0, "Write a Science/Technology News":1}
df_train['prompt'] = df_train['label'].map(label_map)
test_set['prompt'] = test_set['label'].map(label_map)

In [ ]:
df_train

,text,label,prompt
0,Oil prices hit lowest level in 7 weeks Crude o...,0,Write a Business News
1,FCC frees VoIP service from state regulations ...,1,Write a Science/Technology News
2,Human gland evolved from gills The human parat...,1,Write a Science/Technology News
3,A bargain hunters paradise Massachusetts barga...,0,Write a Business News
4,Walking in the Void of Space NASA -- Venturing...,1,Write a Science/Technology News
...,...,...,...
3995,Hollywoods Lion Kings All the morning headline...,1,Write a Science/Technology News
3996,"Carriers, handset makers vie for interface con...",1,Write a Science/Technology News
3997,Wal-Mart Clouds November Retail Investors floc...,0,Write a Business News
3998,Cisco warns of IOS flaw Cisco Systems Inc. war...,1,Write a Science/Technology News


## BART

In [ ]:
from torch.utils.data import Dataset

class TextDataset(Dataset):
    def __init__(self, tokenizer, df, max_length=512):
        self.tokenizer = tokenizer
        self.inputs = df['prompt']
        self.targets = df['text']
        self.max_length = max_length

    def __len__(self):
        return len(self.inputs)

    def __getitem__(self, idx):
        input_text = self.inputs[idx]
        target_text = self.targets[idx]

        # Tokenize input and target text
        input_encoding = self.tokenizer(input_text, max_length=self.max_length, truncation=True, padding='max_length', return_tensors='pt')
        target_encoding = self.tokenizer(target_text, max_length=self.max_length, truncation=True, padding='max_length', return_tensors='pt')

        return {'input_ids': input_encoding['input_ids'].flatten(),
                'attention_mask': input_encoding['attention_mask'].flatten(),
                'labels': target_encoding['input_ids'].flatten()}


In [ ]:
from transformers import BartTokenizer, BartForConditionalGeneration
tokenizer = BartTokenizer.from_pretrained("facebook/bart-base")
model = BartForConditionalGeneration.from_pretrained("facebook/bart-base")

In [ ]:
train1 = df_train[df_train['label']==1].reset_index(drop=True)
test1 = test_set[test_set['label']==1].reset_index(drop=True)

In [ ]:
df_train

,text,label,prompt
0,Oil prices hit lowest level in 7 weeks Crude o...,0,Write a Business News
1,FCC frees VoIP service from state regulations ...,1,Write a Science/Technology News
2,Human gland evolved from gills The human parat...,1,Write a Science/Technology News
3,A bargain hunters paradise Massachusetts barga...,0,Write a Business News
4,Walking in the Void of Space NASA -- Venturing...,1,Write a Science/Technology News
...,...,...,...
3995,Hollywoods Lion Kings All the morning headline...,1,Write a Science/Technology News
3996,"Carriers, handset makers vie for interface con...",1,Write a Science/Technology News
3997,Wal-Mart Clouds November Retail Investors floc...,0,Write a Business News
3998,Cisco warns of IOS flaw Cisco Systems Inc. war...,1,Write a Science/Technology News


In [ ]:
from torch.utils.data import DataLoader
train_dataset = TextDataset(tokenizer, train1,max_length=320)
train_loader = DataLoader(train_dataset, batch_size=16)
val_dataset = TextDataset(tokenizer, test1,max_length=320)
val_loader = DataLoader(val_dataset, batch_size=16)

In [ ]:
import torch
import pandas as pd
import random
import numpy as np

def set_seeds(seed):
    torch.manual_seed(seed)
    random.seed(seed)
    np.random.seed(seed)

def generate_text_with_beams(prompt, num_texts=5, temperature=1.0, top_k=50, top_p=0.95, min_length=50, max_length=320, num_beams=5, no_repeat_ngram_size=2, early_stopping=True, seed=42, repetition_penalty=1.5):
    if seed is not None:
        set_seeds(seed)
    model.eval()
    generated_texts = []
    for _ in range(num_texts):
        # Encode the prompt to generate input_ids and attention_mask
        encoding = tokenizer(prompt, return_tensors='pt', padding='max_length', max_length=max_length, truncation=True)
        input_ids = encoding['input_ids'].to(model.device)
        attention_mask = encoding['attention_mask'].to(model.device)

        outputs = model.generate(
            input_ids,
            attention_mask=attention_mask,  # Include attention mask in generation
            min_length=min_length,
            max_length=max_length,
            num_beams=num_beams,
            num_return_sequences=1,  # Generate one sequence at a time to ensure diversity
            temperature=temperature,
            top_k=top_k,
            top_p=top_p,
            no_repeat_ngram_size=no_repeat_ngram_size,
            repetition_penalty=repetition_penalty,
            do_sample=True,
            pad_token_id=tokenizer.eos_token_id,
            early_stopping=early_stopping
        )

        text = tokenizer.decode(outputs[0], skip_special_tokens=True)
        generated_texts.append(text)

    return generated_texts

def generate_text(top_p, top_k, temperature, num_beams=5, num_texts=20, seed=42, repetition_penalty=1.2):
    prompts = ["Write a Science/Technology News"]
    all_prompts = []
    all_texts = []

    for prompt in prompts:
        generated_texts_for_prompt = []
        num_iterations = num_texts // num_beams

        for i in range(num_iterations):
            set_seeds(seed + i)
            generated_texts = generate_text_with_beams(
                prompt=prompt,
                num_texts=num_beams,
                temperature=temperature,
                top_k=top_k,
                top_p=top_p,
                min_length=50,
                max_length=200,  # Adjust max_length as needed
                num_beams=num_beams,
                no_repeat_ngram_size=2,
                early_stopping=True,  # Enable early stopping
                seed=seed + i,  # Adjust seed for each batch to maintain diversity while being reproducible
                repetition_penalty=repetition_penalty  # Adding repetition penalty
            )
            generated_texts_for_prompt.extend(generated_texts)

        # Ensure uniqueness and limit to the desired number of texts
        unique_texts = list(set(generated_texts_for_prompt))[:num_texts]
        all_prompts.extend([prompt] * len(unique_texts))
        all_texts.extend(unique_texts)

    return pd.DataFrame({'prompt': all_prompts, 'text': all_texts})


In [ ]:
import numpy as np
import torch
from transformers import BertTokenizer, BertModel
from scipy.linalg import sqrtm
import pandas as pd

# Load BERT model and tokenizer
bert_tokenizer_base = BertTokenizer.from_pretrained('bert-base-uncased')
bert_model_base = BertModel.from_pretrained('bert-base-uncased')
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
bert_model_base.to(device)
# Function to get embeddings
def get_embeddings(texts, batch_size=32):
    all_embeddings = []
    for i in range(0, len(texts), batch_size):
        batch_texts = texts[i:i+batch_size]
        inputs = bert_tokenizer_base(batch_texts, return_tensors='pt', padding=True, truncation=True, max_length=320)
        inputs = {k: v.to(device) for k, v in inputs.items()}
        with torch.no_grad():
            outputs = bert_model_base(**inputs)
        embeddings = outputs.last_hidden_state.mean(dim=1)
        all_embeddings.append(embeddings.cpu().numpy())
        torch.cuda.empty_cache()  # Clear unused memory
    return np.vstack(all_embeddings)

# Function to calculate mean and covariance
def calculate_mean_covariance(embeddings):
    mu = np.mean(embeddings, axis=0)
    sigma = np.cov(embeddings, rowvar=False)
    return mu, sigma

# Function to compute Fréchet Distance
def calculate_frechet_distance(mu1, sigma1, mu2, sigma2):
    diff = mu1 - mu2
    covmean, _ = sqrtm(sigma1.dot(sigma2), disp=False)
    if np.iscomplexobj(covmean):
        covmean = covmean.real
    return np.sum(diff**2) + np.trace(sigma1 + sigma2 - 2*covmean)

In [ ]:
df_original = df_train[df_train['label']==1].sample(n=200, random_state=13)
original_texts = df_original['text'].tolist()
original_embeddings = get_embeddings(original_texts)
mu_orig, sigma_orig = calculate_mean_covariance(original_embeddings)

In [ ]:
## modified on April 1st
from transformers import AdamW
from tqdm.auto import tqdm
import torch
import os
device = 'cuda' if torch.cuda.is_available() else 'cpu'
model.to(device)
optimizer = torch.optim.AdamW(model.parameters(), lr=1e-5)
num_epochs = 70
model.train()

# Initialize a folder path for saving models
folder_path = "/content/drive/My Drive"

best_fid = float('inf')
for epoch in range(num_epochs):
    total_loss = 0
    total_val_loss = 0

    # Training
    model.train()
    for batch in tqdm(train_loader, desc=f'Epoch {epoch+1} Training'):
        optimizer.zero_grad()
        inputs = {k: v.to(device) for k, v in batch.items()}
        outputs = model(**inputs)
        loss = outputs.loss
        loss.backward()
        optimizer.step()

        total_loss += loss.item()

    avg_train_loss = total_loss / len(train_loader)

    # Validation
    model.eval()
    with torch.no_grad():
        for batch in tqdm(val_loader, desc=f'Epoch {epoch+1} Validation'):
            inputs = {k: v.to(device) for k, v in batch.items()}
            outputs = model(**inputs)
            loss = outputs.loss

            total_val_loss += loss.item()

    avg_val_loss = total_val_loss / len(val_loader)
    print(f'Epoch {epoch+1}: Training Loss: {avg_train_loss:.4f}, Validation Loss: {avg_val_loss:.4f}')

    # Generate text and calculate FID
    df_text = generate_text(top_p=0.95, top_k=45, temperature=4.7, num_beams=4, num_texts=200, repetition_penalty=1.7)
    generated_texts = df_text['text'].tolist()
    generated_embeddings = get_embeddings(generated_texts)
    mu_generated, sigma_generated = calculate_mean_covariance(generated_embeddings)
    fid = calculate_frechet_distance(mu_orig, sigma_orig, mu_generated, sigma_generated)
    print(f'Epoch {epoch+1}: FID: {fid:.4f}')

    # Save the model if the current FID is lower than the best FID
    if fid < best_fid:
        best_fid = fid
        save_path = os.path.join(folder_path, f'model_81_bart_class1_epoch_{epoch + 1}')
        model.save_pretrained(save_path)
        tokenizer.save_pretrained(save_path)
        print(f'Model saved to {save_path} with FID: {fid:.4f}')


Epoch 1 Training:   0%|          | 0/125 [00:00<?, ?it/s]

Epoch 1 Validation:   0%|          | 0/63 [00:00<?, ?it/s]

Epoch 1: Training Loss: 7.4331, Validation Loss: 2.9317


Some non-default generation parameters are set in the model config. These should go into a GenerationConfig file (https://huggingface.co/docs/transformers/generation_strategies#save-a-custom-decoding-strategy-with-your-model) instead. This warning will be raised to an exception in v4.41.
Non-default generation parameters: {'early_stopping': True, 'num_beams': 4, 'no_repeat_ngram_size': 3, 'forced_bos_token_id': 0, 'forced_eos_token_id': 2}


Epoch 1: FID: 10.4342
Model saved to /content/drive/My Drive/model_81_bart_class1_epoch_1 with FID: 10.4342


...

Some non-default generation parameters are set in the model config. These should go into a GenerationConfig file (https://huggingface.co/docs/transformers/generation_strategies#save-a-custom-decoding-strategy-with-your-model) instead. This warning will be raised to an exception in v4.41.
Non-default generation parameters: {'early_stopping': True, 'num_beams': 4, 'no_repeat_ngram_size': 3, 'forced_bos_token_id': 0, 'forced_eos_token_id': 2}


Epoch 69: FID: 7.4286
Model saved to /content/drive/My Drive/model_81_bart_class1_epoch_69 with FID: 7.4286


Epoch 70 Training:   0%|          | 0/125 [00:00<?, ?it/s]

Epoch 70 Validation:   0%|          | 0/63 [00:00<?, ?it/s]

Epoch 70: Training Loss: 0.2562, Validation Loss: 0.7038
Epoch 70: FID: 8.2207


## test0 81

In [ ]:
from transformers import BartTokenizer, BartForConditionalGeneration
tokenizer = BartTokenizer.from_pretrained("/content/drive/MyDrive/model_81_bart_class1_epoch_69")
model = BartForConditionalGeneration.from_pretrained("/content/drive/MyDrive/model_81_bart_class1_epoch_69")

In [ ]:
import pandas as pd
import torch
device = 'cuda' if torch.cuda.is_available() else 'cpu'
model.eval()
model.to(device)

BartForConditionalGeneration(
  (model): BartModel(
    (shared): Embedding(50265, 768, padding_idx=1)
    (encoder): BartEncoder(
      (embed_tokens): BartScaledWordEmbedding(50265, 768, padding_idx=1)
      (embed_positions): BartLearnedPositionalEmbedding(1026, 768)
      (layers): ModuleList(
        (0-5): 6 x BartEncoderLayer(
          (self_attn): BartSdpaAttention(
            (k_proj): Linear(in_features=768, out_features=768, bias=True)
            (v_proj): Linear(in_features=768, out_features=768, bias=True)
            (q_proj): Linear(in_features=768, out_features=768, bias=True)
            (out_proj): Linear(in_features=768, out_features=768, bias=True)
          )
          (self_attn_layer_norm): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
          (activation_fn): GELUActivation()
          (fc1): Linear(in_features=768, out_features=3072, bias=True)
          (fc2): Linear(in_features=3072, out_features=768, bias=True)
          (final_layer_norm): Laye

In [ ]:
import torch
import pandas as pd
import random
import numpy as np

def set_seeds(seed):
    torch.manual_seed(seed)
    random.seed(seed)
    np.random.seed(seed)

def generate_text_with_beams(prompt, num_texts=5, temperature=1.0, top_k=50, top_p=0.95, min_length=50, max_length=320, num_beams=5, no_repeat_ngram_size=2, early_stopping=True, seed=42, repetition_penalty=1.5):
    if seed is not None:
        set_seeds(seed)
    model.eval()
    generated_texts = []
    for _ in range(num_texts):
        # Encode the prompt to generate input_ids and attention_mask
        encoding = tokenizer(prompt, return_tensors='pt', padding='max_length', max_length=max_length, truncation=True)
        input_ids = encoding['input_ids'].to(model.device)
        attention_mask = encoding['attention_mask'].to(model.device)

        outputs = model.generate(
            input_ids,
            attention_mask=attention_mask,  # Include attention mask in generation
            min_length=min_length,
            max_length=max_length,
            num_beams=num_beams,
            num_return_sequences=1,  # Generate one sequence at a time to ensure diversity
            temperature=temperature,
            top_k=top_k,
            top_p=top_p,
            no_repeat_ngram_size=no_repeat_ngram_size,
            repetition_penalty=repetition_penalty,
            do_sample=True,
            pad_token_id=tokenizer.eos_token_id,
            early_stopping=early_stopping
        )

        text = tokenizer.decode(outputs[0], skip_special_tokens=True)
        generated_texts.append(text)

    return generated_texts

def generate_text(top_p, top_k, temperature, num_beams=5, num_texts=20, seed=42, repetition_penalty=1.2):
    prompts = [ "Write a Science/Technology News"]
    all_prompts = []
    all_texts = []

    for prompt in prompts:
        generated_texts_for_prompt = []
        num_iterations = num_texts // num_beams

        for i in range(num_iterations):
            set_seeds(seed + i)
            generated_texts = generate_text_with_beams(
                prompt=prompt,
                num_texts=num_beams,
                temperature=temperature,
                top_k=top_k,
                top_p=top_p,
                min_length=50,
                max_length=200,  # Adjust max_length as needed
                num_beams=num_beams,
                no_repeat_ngram_size=2,
                early_stopping=True,  # Enable early stopping
                seed=seed + i,  # Adjust seed for each batch to maintain diversity while being reproducible
                repetition_penalty=repetition_penalty  # Adding repetition penalty
            )
            generated_texts_for_prompt.extend(generated_texts)

        # Ensure uniqueness and limit to the desired number of texts
        unique_texts = list(set(generated_texts_for_prompt))[:num_texts]
        all_prompts.extend([prompt] * len(unique_texts))
        all_texts.extend(unique_texts)

    return pd.DataFrame({'prompt': all_prompts, 'text': all_texts})


In [ ]:
import torch
from torch.utils.data import Dataset, DataLoader
class TextDataset(Dataset):
    def __init__(self, dataframe, tokenizer, max_len=320):
        self.tokenizer = tokenizer
        self.data = dataframe
        self.text = dataframe.text
        self.targets = dataframe.label
        self.max_len = max_len

    def __len__(self):
        return len(self.text)

    def __getitem__(self, index):
        text = str(self.text[index])
        text = " ".join(text.split())

        inputs = self.tokenizer.encode_plus(
            text,
            None,
            add_special_tokens=True,
            max_length=self.max_len,
            padding='max_length',
            truncation=True,
            return_attention_mask=True,
            return_tensors="pt",
        )

        return {
            'input_ids': inputs['input_ids'].flatten(),
            'attention_mask': inputs['attention_mask'].flatten(),
            'labels': torch.tensor(self.targets[index], dtype=torch.long)
        }


In [ ]:
from transformers import BertTokenizer, BertForSequenceClassification
import torch
device = 'cuda' if torch.cuda.is_available() else 'cpu'
from sklearn.metrics import accuracy_score
import numpy as np
from collections import defaultdict
from transformers import BertTokenizer
from transformers import BertForSequenceClassification
bert_tokenizer = BertTokenizer.from_pretrained('/content/drive/MyDrive/planB/bert/BERT_baseline_723_2')
bert_model = BertForSequenceClassification.from_pretrained('/content/drive/MyDrive/planB/bert/BERT_baseline_723_2', num_labels=2)
bert_model.to(device)
def evaluate(model, dataloader):
    model.eval()

    predictions, true_labels = [], []

    with torch.no_grad():
        for batch in dataloader:
            input_ids = batch['input_ids'].to(device)
            attention_mask = batch['attention_mask'].to(device)
            labels = batch['labels'].to(device)

            outputs = model(input_ids, attention_mask=attention_mask, labels=labels)

            logits = outputs.logits.detach().cpu().numpy()
            label_ids = labels.to('cpu').numpy()

            predictions.extend(np.argmax(logits, axis=1).flatten())
            true_labels.extend(label_ids.flatten())

    avg_accuracy = accuracy_score(true_labels, predictions)

    # Calculate accuracy for each class
    class_accuracies = defaultdict(list)
    for true, pred in zip(true_labels, predictions):
        class_accuracies[true].append(pred == true)

    class_accuracy_results = {label: np.mean(acc) for label, acc in class_accuracies.items()}

    return avg_accuracy, class_accuracy_results

In [ ]:
!pip install sacrebleu

import pandas as pd
import numpy as np
import sacrebleu

def calculate_self_bleu(texts):
    scores = []
    for i, target in enumerate(texts):
        references = texts[:i] + texts[i+1:]
        bleu_score = sacrebleu.corpus_bleu([target], [references]).score
        scores.append(bleu_score)
    return np.mean(scores)

def calculate_self_bleu_per_class(df):
    class_bleu_scores = {}
    for label, group in df.groupby('label'):
        texts = group['text'].tolist()
        self_bleu_score = calculate_self_bleu(texts)
        class_bleu_scores[label] = self_bleu_score
    return class_bleu_scores

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.0/58.0 kB 2.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 106.7/106.7 kB 4.6 MB/s eta 0:00:00


In [ ]:
import pandas as pd
from sklearn.metrics import jaccard_score
from itertools import combinations
import numpy as np

# Function to calculate Jaccard Similarity between two texts
def jaccard_similarity(text1, text2):
    set1 = set(text1.split())
    set2 = set(text2.split())
    intersection = set1.intersection(set2)
    union = set1.union(set2)
    return len(intersection) / len(union) if len(union) > 0 else 0

# Function to calculate mean Jaccard similarity within each class
def calculate_self_jaccard_similarity(df):
    results = {}
    for label, group in df.groupby('label'):
        jaccard_similarities = [jaccard_similarity(text1, text2) for text1, text2 in combinations(group['text'], 2)]
        results[label] = np.mean(jaccard_similarities) if jaccard_similarities else 0
    return results

In [ ]:
label_map2 = {"Write a Business News":0, "Write a Science/Technology News":1}

In [ ]:
import numpy as np
import torch
from transformers import BertTokenizer, BertModel
from scipy.linalg import sqrtm
import pandas as pd

# Load BERT model and tokenizer
bert_tokenizer_base = BertTokenizer.from_pretrained('bert-base-uncased')
bert_model_base = BertModel.from_pretrained('bert-base-uncased')
bert_model_base.to(device)
def get_embeddings(texts, batch_size=32):
    all_embeddings = []
    for i in range(0, len(texts), batch_size):
        batch_texts = texts[i:i+batch_size]
        inputs = bert_tokenizer_base(batch_texts, return_tensors='pt', padding=True, truncation=True, max_length=320)
        inputs = {k: v.to(device) for k, v in inputs.items()}
        with torch.no_grad():
            outputs = bert_model_base(**inputs)
        embeddings = outputs.last_hidden_state.mean(dim=1)
        all_embeddings.append(embeddings.cpu().numpy())
        torch.cuda.empty_cache()  # Clear unused memory
    return np.vstack(all_embeddings)

# Function to calculate mean and covariance
def calculate_mean_covariance(embeddings):
    mu = np.mean(embeddings, axis=0)
    sigma = np.cov(embeddings, rowvar=False)
    return mu, sigma

# Function to compute Fréchet Distance
def calculate_frechet_distance(mu1, sigma1, mu2, sigma2):
    diff = mu1 - mu2
    covmean, _ = sqrtm(sigma1.dot(sigma2), disp=False)
    if np.iscomplexobj(covmean):
        covmean = covmean.real
    return np.sum(diff**2) + np.trace(sigma1 + sigma2 - 2*covmean)

tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

config.json:   0%|          | 0.00/570 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/440M [00:00<?, ?B/s]

In [ ]:
df_original0 = df_train[df_train['label']==0].sample(n=500, random_state=13)
original_texts0 = df_original0['text'].tolist()
original_embeddings0 = get_embeddings(original_texts0)
df_original1 = df_train[df_train['label']==1].sample(n=500, random_state=13)
original_texts1 = df_original1['text'].tolist()
original_embeddings1 = get_embeddings(original_texts1)
mu_orig0, sigma_orig0 = calculate_mean_covariance(original_embeddings0)
mu_orig1, sigma_orig1 = calculate_mean_covariance(original_embeddings1)

In [ ]:
for k in [40,45,50]:
  df_text = generate_text(top_p=0.95, top_k=k, temperature=3.7, num_beams=4 , num_texts=500, repetition_penalty=1.7,seed=724)
  df_text['label'] = df_text['prompt'].map(label_map2)
  dataset = TextDataset(dataframe=df_text, tokenizer=bert_tokenizer, max_len=320)
  loader = DataLoader(dataset, batch_size=16, shuffle=False)
  bert_acc = evaluate(bert_model, loader)
  js = calculate_self_jaccard_similarity(df_text)
  sb = calculate_self_bleu_per_class(df_text)
  df_generate1 = df_text[df_text['label']==1]
  generated_texts1 = df_generate1['text'].tolist()
  generated_embeddings1 = get_embeddings(generated_texts1)
  mu_gen1, sigma_gen1 = calculate_mean_covariance(generated_embeddings1)

  fid1 = calculate_frechet_distance(mu_orig1, sigma_orig1, mu_gen1, sigma_gen1)
  print(f"k: {k}")
  print(f"bert accuracy: {bert_acc}")
  print(f"Jaccard similarities: {js}")
  print(f"Self-BLEU: {sb}")
  print(fid1)

k: 40
bert accuracy: (0.982, {1: 0.982})
Jaccard similarities: {1: 0.06905424068533292}
Self-BLEU: {1: 1.2618093839496831}
4.528457425966696
k: 45
bert accuracy: (0.974, {1: 0.974})
Jaccard similarities: {1: 0.06741114920370249}
Self-BLEU: {1: 0.9825441296337595}
4.145145268015739
k: 50
bert accuracy: (0.98, {1: 0.98})
Jaccard similarities: {1: 0.06604278261311229}
Self-BLEU: {1: 1.308652768326823}
4.034950990947852


In [ ]:
for k in [40,45,50]:
  df_text = generate_text(top_p=0.95, top_k=k, temperature=4.1, num_beams=4 , num_texts=500, repetition_penalty=1.7,seed=724)
  df_text['label'] = df_text['prompt'].map(label_map2)
  dataset = TextDataset(dataframe=df_text, tokenizer=bert_tokenizer, max_len=320)
  loader = DataLoader(dataset, batch_size=16, shuffle=False)
  bert_acc = evaluate(bert_model, loader)
  js = calculate_self_jaccard_similarity(df_text)
  sb = calculate_self_bleu_per_class(df_text)
  df_generate1 = df_text[df_text['label']==1]
  generated_texts1 = df_generate1['text'].tolist()
  generated_embeddings1 = get_embeddings(generated_texts1)
  mu_gen1, sigma_gen1 = calculate_mean_covariance(generated_embeddings1)

  fid1 = calculate_frechet_distance(mu_orig1, sigma_orig1, mu_gen1, sigma_gen1)
  print(f"k: {k}")
  print(f"bert accuracy: {bert_acc}")
  print(f"Jaccard similarities: {js}")
  print(f"Self-BLEU: {sb}")
  print(fid1)
  ## this one

k: 40
bert accuracy: (0.974, {1: 0.974})
Jaccard similarities: {1: 0.06677845700880919}
Self-BLEU: {1: 1.2460879307284523}
4.21607277198295
k: 45
bert accuracy: (0.968, {1: 0.968})
Jaccard similarities: {1: 0.06515656792848495}
Self-BLEU: {1: 1.1321612598196298}
3.9086340515161266
k: 50
bert accuracy: (0.978, {1: 0.978})
Jaccard similarities: {1: 0.06547116336171306}
Self-BLEU: {1: 1.2254141397931715}
4.01924765363944


In [ ]:
for k in [40,45,50]:
  df_text = generate_text(top_p=0.95, top_k=k, temperature=4.3, num_beams=4 , num_texts=500, repetition_penalty=1.7,seed=724)
  df_text['label'] = df_text['prompt'].map(label_map2)
  dataset = TextDataset(dataframe=df_text, tokenizer=bert_tokenizer, max_len=320)
  loader = DataLoader(dataset, batch_size=16, shuffle=False)
  bert_acc = evaluate(bert_model, loader)
  js = calculate_self_jaccard_similarity(df_text)
  sb = calculate_self_bleu_per_class(df_text)
  df_generate1 = df_text[df_text['label']==1]
  generated_texts1 = df_generate1['text'].tolist()
  generated_embeddings1 = get_embeddings(generated_texts1)
  mu_gen1, sigma_gen1 = calculate_mean_covariance(generated_embeddings1)

  fid1 = calculate_frechet_distance(mu_orig1, sigma_orig1, mu_gen1, sigma_gen1)
  print(f"k: {k}")
  print(f"bert accuracy: {bert_acc}")
  print(f"Jaccard similarities: {js}")
  print(f"Self-BLEU: {sb}")
  print(fid1)
  ## this one

k: 40
bert accuracy: (0.976, {1: 0.976})
Jaccard similarities: {1: 0.06777460675994189}
Self-BLEU: {1: 1.0347257314152107}
4.201559787984857
k: 45
bert accuracy: (0.978, {1: 0.978})
Jaccard similarities: {1: 0.06382296294021828}
Self-BLEU: {1: 1.261064577820559}
4.049706154106106
k: 50
bert accuracy: (0.974, {1: 0.974})
Jaccard similarities: {1: 0.06479205966420125}
Self-BLEU: {1: 1.2295038612432556}
4.206918834495475


In [ ]:
for k in [40,45,50]:
  df_text = generate_text(top_p=0.95, top_k=k, temperature=4.5, num_beams=4 , num_texts=500, repetition_penalty=1.7,seed=724)
  df_text['label'] = df_text['prompt'].map(label_map2)
  dataset = TextDataset(dataframe=df_text, tokenizer=bert_tokenizer, max_len=320)
  loader = DataLoader(dataset, batch_size=16, shuffle=False)
  bert_acc = evaluate(bert_model, loader)
  js = calculate_self_jaccard_similarity(df_text)
  sb = calculate_self_bleu_per_class(df_text)
  df_generate1 = df_text[df_text['label']==1]
  generated_texts1 = df_generate1['text'].tolist()
  generated_embeddings1 = get_embeddings(generated_texts1)
  mu_gen1, sigma_gen1 = calculate_mean_covariance(generated_embeddings1)

  fid1 = calculate_frechet_distance(mu_orig1, sigma_orig1, mu_gen1, sigma_gen1)
  print(f"k: {k}")
  print(f"bert accuracy: {bert_acc}")
  print(f"Jaccard similarities: {js}")
  print(f"Self-BLEU: {sb}")
  print(fid1)
  ## this one

k: 40
bert accuracy: (0.984, {1: 0.984})
Jaccard similarities: {1: 0.06719142625516006}
Self-BLEU: {1: 1.1579028711069008}
4.1012299231212905
k: 45
bert accuracy: (0.976, {1: 0.976})
Jaccard similarities: {1: 0.06499143098294805}
Self-BLEU: {1: 1.2068144332877917}
4.110958950527063
k: 50
bert accuracy: (0.972, {1: 0.972})
Jaccard similarities: {1: 0.06407770616980914}
Self-BLEU: {1: 1.0333657873675492}
3.9573774498765015


In [ ]:
for k in [40,45,50]:
  df_text = generate_text(top_p=0.95, top_k=k, temperature=4.7, num_beams=4 , num_texts=500, repetition_penalty=1.7,seed=724)
  df_text['label'] = df_text['prompt'].map(label_map2)
  dataset = TextDataset(dataframe=df_text, tokenizer=bert_tokenizer, max_len=320)
  loader = DataLoader(dataset, batch_size=16, shuffle=False)
  bert_acc = evaluate(bert_model, loader)
  js = calculate_self_jaccard_similarity(df_text)
  sb = calculate_self_bleu_per_class(df_text)
  df_generate1 = df_text[df_text['label']==1]
  generated_texts1 = df_generate1['text'].tolist()
  generated_embeddings1 = get_embeddings(generated_texts1)
  mu_gen1, sigma_gen1 = calculate_mean_covariance(generated_embeddings1)

  fid1 = calculate_frechet_distance(mu_orig1, sigma_orig1, mu_gen1, sigma_gen1)
  print(f"k: {k}")
  print(f"bert accuracy: {bert_acc}")
  print(f"Jaccard similarities: {js}")
  print(f"Self-BLEU: {sb}")
  print(fid1)
  ## this one

k: 40
bert accuracy: (0.978, {1: 0.978})
Jaccard similarities: {1: 0.06515876035785811}
Self-BLEU: {1: 1.2911546392295}
3.969484354105888
k: 45
bert accuracy: (0.98, {1: 0.98})
Jaccard similarities: {1: 0.06464754739484214}
Self-BLEU: {1: 0.7376890022550678}
4.116566339151534
k: 50
bert accuracy: (0.98, {1: 0.98})
Jaccard similarities: {1: 0.06341837996034376}
Self-BLEU: {1: 1.1254275573551853}
4.160562725089627


In [ ]:
## not yet
for rp in [1.3,1.9,2.1]:
  df_text = generate_text(top_p=0.95, top_k=40, temperature=4.7, num_beams=4 , num_texts=500, repetition_penalty=rp,seed=724)
  df_text['label'] = df_text['prompt'].map(label_map2)
  dataset = TextDataset(dataframe=df_text, tokenizer=bert_tokenizer, max_len=320)
  loader = DataLoader(dataset, batch_size=16, shuffle=False)
  bert_acc = evaluate(bert_model, loader)
  js = calculate_self_jaccard_similarity(df_text)
  sb = calculate_self_bleu_per_class(df_text)
  df_generate1 = df_text[df_text['label']==1]
  generated_texts1 = df_generate1['text'].tolist()
  generated_embeddings1 = get_embeddings(generated_texts1)
  mu_gen1, sigma_gen1 = calculate_mean_covariance(generated_embeddings1)

  fid1 = calculate_frechet_distance(mu_orig1, sigma_orig1, mu_gen1, sigma_gen1)
  print(f"k: {rp}")
  print(f"bert accuracy: {bert_acc}")
  print(f"Jaccard similarities: {js}")
  print(f"Self-BLEU: {sb}")
  print(fid1)

k: 1.3
bert accuracy: (0.978, {1: 0.978})
Jaccard similarities: {1: 0.06436817814190998}
Self-BLEU: {1: 1.0335621377771533}
3.983130285124938
k: 1.9
bert accuracy: (0.966, {1: 0.966})
Jaccard similarities: {1: 0.06494181827478426}
Self-BLEU: {1: 1.1049266894539127}
4.005199031451172
k: 2.1
bert accuracy: (0.974, {1: 0.974})
Jaccard similarities: {1: 0.06686724003482773}
Self-BLEU: {1: 1.6334852401078797}
3.993143772751442


In [ ]:
## not yet
for rp in [1.3,1.9,2.1]:
  df_text = generate_text(top_p=0.95, top_k=50, temperature=4.5, num_beams=4 , num_texts=500, repetition_penalty=rp,seed=724)
  df_text['label'] = df_text['prompt'].map(label_map2)
  dataset = TextDataset(dataframe=df_text, tokenizer=bert_tokenizer, max_len=320)
  loader = DataLoader(dataset, batch_size=16, shuffle=False)
  bert_acc = evaluate(bert_model, loader)
  js = calculate_self_jaccard_similarity(df_text)
  sb = calculate_self_bleu_per_class(df_text)
  df_generate1 = df_text[df_text['label']==1]
  generated_texts1 = df_generate1['text'].tolist()
  generated_embeddings1 = get_embeddings(generated_texts1)
  mu_gen1, sigma_gen1 = calculate_mean_covariance(generated_embeddings1)

  fid1 = calculate_frechet_distance(mu_orig1, sigma_orig1, mu_gen1, sigma_gen1)
  print(f"k: {rp}")
  print(f"bert accuracy: {bert_acc}")
  print(f"Jaccard similarities: {js}")
  print(f"Self-BLEU: {sb}")
  print(fid1)

k: 1.3
bert accuracy: (0.982, {1: 0.982})
Jaccard similarities: {1: 0.06291394323922364}
Self-BLEU: {1: 0.6403436380488399}
4.094368004159426
k: 1.9
bert accuracy: (0.978, {1: 0.978})
Jaccard similarities: {1: 0.06342829113441058}
Self-BLEU: {1: 1.1589391626418732}
4.022096147514948
k: 2.1
bert accuracy: (0.98, {1: 0.98})
Jaccard similarities: {1: 0.0653966658348115}
Self-BLEU: {1: 1.1532964799971186}
4.169384433440859


In [ ]:
df_text = generate_text(top_p=0.95, top_k=45, temperature=4.1, num_beams=4 , num_texts=10000, repetition_penalty=1.7,seed=81)
df_text['label'] = df_text['prompt'].map(label_map2)
df_text.to_csv("/content/drive/MyDrive/planB/df_generated_bart_class1_81_0.csv")

In [ ]:
df_text = generate_text(top_p=0.95, top_k=50, temperature=4.1, num_beams=4 , num_texts=10000, repetition_penalty=1.7,seed=81)
df_text['label'] = df_text['prompt'].map(label_map2)
df_text.to_csv("/content/drive/MyDrive/planB/df_generated_bart_class1_81_1.csv")

In [ ]:
df_text = generate_text(top_p=0.95, top_k=45, temperature=4.3, num_beams=4 , num_texts=10000, repetition_penalty=1.7,seed=81)
df_text['label'] = df_text['prompt'].map(label_map2)
df_text.to_csv("/content/drive/MyDrive/planB/df_generated_bart_class1_81_2.csv")

In [ ]:
df_text = generate_text(top_p=0.95, top_k=45, temperature=4.3, num_beams=4 , num_texts=10000, repetition_penalty=1.7,seed=87)
df_text['label'] = df_text['prompt'].map(label_map2)
df_text.to_csv("/content/drive/MyDrive/planB/df_generated_bart_class1_81_22.csv")

In [ ]:
df_text = generate_text(top_p=0.95, top_k=40, temperature=4.7, num_beams=4 , num_texts=10000, repetition_penalty=1.7,seed=84)
df_text['label'] = df_text['prompt'].map(label_map2)
df_text.to_csv("/content/drive/MyDrive/planB/df_generated_bart_class1_81_3.csv")

In [ ]:
df_text = generate_text(top_p=0.95, top_k=40, temperature=4.7, num_beams=4 , num_texts=10000, repetition_penalty=1.7,seed=87)
df_text['label'] = df_text['prompt'].map(label_map2)
df_text.to_csv("/content/drive/MyDrive/planB/df_generated_bart_class1_81_33.csv")

KeyboardInterrupt: 

In [ ]:
df_text = generate_text(top_p=0.95, top_k=40, temperature=4.7, num_beams=4 , num_texts=10000, repetition_penalty=1.7,seed=87)
df_text['label'] = df_text['prompt'].map(label_map2)
df_text.to_csv("/content/drive/MyDrive/planB/df_generated_bart_class1_81_32.csv")

In [ ]:
df_text = generate_text(top_p=0.95, top_k=50, temperature=4.5, num_beams=4 , num_texts=10000, repetition_penalty=1.7,seed=84)
df_text['label'] = df_text['prompt'].map(label_map2)
df_text.to_csv("/content/drive/MyDrive/planB/df_generated_bart_class1_81_4.csv")

In [ ]:
print(bert_acc)

(0.978, {1: 0.978})


In [ ]:
df_original0 = df_train[df_train['label']==0]
original_texts0 = df_original0['text'].tolist()
original_embeddings0 = get_embeddings(original_texts0)
df_original1 = df_train[df_train['label']==1]
original_texts1 = df_original1['text'].tolist()
original_embeddings1 = get_embeddings(original_texts1)
mu_orig0, sigma_orig0 = calculate_mean_covariance(original_embeddings0)
mu_orig1, sigma_orig1 = calculate_mean_covariance(original_embeddings1)

In [ ]:
df_text

,prompt,text,label
0,Write a Science/Technology News,"Sasser, Gates Back Capsule With New Microsoft ...",1
1,Write a Science/Technology News,NEC to Buy German PC vendor Intel Corporation ...,1
2,Write a Science/Technology News,Google to launch content management software G...,1
3,Write a Science/Technology News,Microsoft Releases Anti-Spam Product for Augus...,1
4,Write a Science/Technology News,NASA Flights New Space Ship to Test Phones Oct...,1
...,...,...,...
9995,Write a Science/Technology News,"Apple disables iPod juice juice, juice bars Ap...",1
9996,Write a Science/Technology News,Swatch Eyes Blindfold Market - Software giant ...,1
9997,Write a Science/Technology News,Venture Partners Expands DMO Strain Technologi...,1
9998,Write a Science/Technology News,Nvidia 39s Shandong G5 processor combines Inte...,1


In [ ]:
df_text = pd.read_csv("/content/drive/MyDrive/planB/df_generated_bart_class1_81_22.csv")
generated_texts1 = df_text['text'].tolist()
generated_embeddings1 = get_embeddings(generated_texts1)
mu_gen1, sigma_gen1 = calculate_mean_covariance(generated_embeddings1)
fid1 = calculate_frechet_distance(mu_orig1, sigma_orig1, mu_gen1, sigma_gen1)
print(fid1)

2.054204883370948


In [ ]:
df_text1=pd.read_csv("/content/drive/MyDrive/planB/df_generated_bart_class1_1.csv")
dataset = TextDataset(dataframe=df_text1, tokenizer=bert_tokenizer, max_len=320)
loader = DataLoader(dataset, batch_size=16, shuffle=False)
bert_acc = evaluate(bert_model, loader)
js = calculate_self_jaccard_similarity(df_text1)
print(bert_acc)
print(js)

(0.9779, {1: 0.9779})
{1: 0.07319351211725297}


In [ ]:
df_text2=pd.read_csv("/content/drive/MyDrive/planB/df_generated_bart_class1_2.csv")
dataset = TextDataset(dataframe=df_text1, tokenizer=bert_tokenizer, max_len=320)
loader = DataLoader(dataset, batch_size=16, shuffle=False)
bert_acc = evaluate(bert_model, loader)
js = calculate_self_jaccard_similarity(df_text2)
print(bert_acc)
print(js)

(0.9779, {1: 0.9779})
{1: 0.07091136734796893}


In [ ]:
df_original1 = df_train[df_train['label']==1][0:1000]
original_texts1 = df_original1['text'].tolist()
original_embeddings1 = get_embeddings(original_texts1)
mu_orig1, sigma_orig1 = calculate_mean_covariance(original_embeddings1)
df_generate1 = df_train[df_train['label']==1][1000:2000]
generated_texts1 = df_generate1['text'].tolist()
generated_embeddings1 = get_embeddings(generated_texts1)
mu_gen1, sigma_gen1 = calculate_mean_covariance(generated_embeddings1)
fid1 = calculate_frechet_distance(mu_orig1, sigma_orig1, mu_gen1, sigma_gen1)
print(fid1)

1.8417605206148489


In [ ]:
df_original1 = df_train[df_train['label']==1][0:1000]
original_texts1 = df_original1['text'].tolist()
original_embeddings1 = get_embeddings(original_texts1)
mu_orig1, sigma_orig1 = calculate_mean_covariance(original_embeddings1)
df_generate1 = df_train[df_train['label']==0][1000:2000]
generated_texts1 = df_generate1['text'].tolist()
generated_embeddings1 = get_embeddings(generated_texts1)
mu_gen1, sigma_gen1 = calculate_mean_covariance(generated_embeddings1)
fid1 = calculate_frechet_distance(mu_orig1, sigma_orig1, mu_gen1, sigma_gen1)
print(fid1)

8.183885243008145


In [ ]:
df_text=pd.read_csv("/content/drive/MyDrive/planB/df_generated_bart_class1_0.csv")
generated_texts1 = df_text['text'].tolist()
generated_embeddings1 = get_embeddings(generated_texts1)
mu_gen1, sigma_gen1 = calculate_mean_covariance(generated_embeddings1)
fid1 = calculate_frechet_distance(mu_orig1, sigma_orig1, mu_gen1, sigma_gen1)
print(fid1)

3.21268893045666


In [ ]:
df_text=pd.read_csv("/content/drive/MyDrive/planB/df_generated_bart_class1_1.csv")
generated_texts1 = df_text['text'].tolist()
generated_embeddings1 = get_embeddings(generated_texts1)
mu_gen1, sigma_gen1 = calculate_mean_covariance(generated_embeddings1)
fid1 = calculate_frechet_distance(mu_orig1, sigma_orig1, mu_gen1, sigma_gen1)
print(fid1)

3.0926786144048206


In [ ]:
df_text=pd.read_csv("/content/drive/MyDrive/planB/df_generated_bart_class1_2.csv")
generated_texts1 = df_text['text'].tolist()
generated_embeddings1 = get_embeddings(generated_texts1)
mu_gen1, sigma_gen1 = calculate_mean_covariance(generated_embeddings1)
fid1 = calculate_frechet_distance(mu_orig1, sigma_orig1, mu_gen1, sigma_gen1)
print(fid1)

2.782454053963


In [ ]:
df_text

,Unnamed: 0,prompt,text,label
0,0,Write a Science/Technology News,"Oracle, SBC reach Deal to Buy Oracle Business ...",1
1,1,Write a Science/Technology News,"Yahoo Offers Online Music Search, PayPal Reute...",1
2,2,Write a Science/Technology News,Toshiba to unveil notebook memory unit Toshiba...,1
3,3,Write a Science/Technology News,Nvidia Introduces Powerful 3G G5 GPU Nvidia Co...,1
4,4,Write a Science/Technology News,AOL Offers Web-Swap Service Microsoft has intr...,1
...,...,...,...,...
9995,9995,Write a Science/Technology News,NASA 39s Titan Probe lands on Mars NASA offici...,1
9996,9996,Write a Science/Technology News,Microsoft updates Windows XP update Microsoft ...,1
9997,9997,Write a Science/Technology News,Intel Adds Wireless Chips Intel announced that...,1
9998,9998,Write a Science/Technology News,Nvidia To Build New GeForce X800 GPU Nvidia to...,1


In [ ]:
df_text = generate_text(top_p=0.95, top_k=45, temperature=3.3, num_beams=4 , num_texts=10000, repetition_penalty=1.7,seed=729)
df_text['label'] = df_text['prompt'].map(label_map2)
df_text.to_csv("/content/drive/MyDrive/planB/df_generated_bart_class1_1.csv")

In [ ]:
df_text = generate_text(top_p=0.95, top_k=45, temperature=3.7, num_beams=4 , num_texts=10000, repetition_penalty=1.7,seed=729)
df_text['label'] = df_text['prompt'].map(label_map2)
df_text.to_csv("/content/drive/MyDrive/planB/df_generated_bart_class1_2.csv")

In [ ]:
df_text = generate_text(top_p=0.95, top_k=45, temperature=4.1, num_beams=4 , num_texts=10000, repetition_penalty=1.7,seed=729)
df_text['label'] = df_text['prompt'].map(label_map2)
df_text.to_csv("/content/drive/MyDrive/planB/df_generated_bart_class1_3.csv")

In [ ]:
df_text = generate_text(top_p=0.95, top_k=45, temperature=4.1, num_beams=4 , num_texts=10000, repetition_penalty=1.7,seed=87)
df_text['label'] = df_text['prompt'].map(label_map2)
df_text.to_csv("/content/drive/MyDrive/planB/df_generated_bart_class1_33.csv")

In [ ]:
dataset = TextDataset(dataframe=df_text, tokenizer=bert_tokenizer, max_len=320)
loader = DataLoader(dataset, batch_size=16, shuffle=False)
bert_acc = evaluate(bert_model, loader)
#js = calculate_self_jaccard_similarity(df_text)
print(bert_acc)

(0.9788, {1: 0.9788})


In [ ]:
generated_texts1 = df_text['text'].tolist()
generated_embeddings1 = get_embeddings(generated_texts1)
mu_gen1, sigma_gen1 = calculate_mean_covariance(generated_embeddings1)
fid1 = calculate_frechet_distance(mu_orig1, sigma_orig1, mu_gen1, sigma_gen1)
print(fid1)

2.0490165905529873


In [ ]:
df_text = generate_text(top_p=0.95, top_k=45, temperature=4.7, num_beams=4 , num_texts=10000, repetition_penalty=1.7,seed=729)
df_text['label'] = df_text['prompt'].map(label_map2)
df_text.to_csv("/content/drive/MyDrive/planB/df_generated_bart_class1_4.csv")

In [ ]:
df_text = generate_text(top_p=0.95, top_k=50, temperature=4.7, num_beams=4 , num_texts=10000, repetition_penalty=1.7,seed=729)
df_text['label'] = df_text['prompt'].map(label_map2)
df_text.to_csv("/content/drive/MyDrive/planB/df_generated_bart_class1_5.csv")

In [ ]:
df_original0 = df_train[df_train['label']==0]
original_texts0 = df_original0['text'].tolist()
original_embeddings0 = get_embeddings(original_texts0)
mu_orig0, sigma_orig0 = calculate_mean_covariance(original_embeddings0)
df_original1 = df_train[df_train['label']==1]
original_texts1 = df_original1['text'].tolist()
original_embeddings1 = get_embeddings(original_texts1)
mu_orig1, sigma_orig1 = calculate_mean_covariance(original_embeddings1)

In [ ]:
df_text=pd.read_csv("/content/drive/MyDrive/planB/df_generated_bart_class1_5.csv")
generated_texts1 = df_text['text'].tolist()
generated_embeddings1 = get_embeddings(generated_texts1)
mu_gen1, sigma_gen1 = calculate_mean_covariance(generated_embeddings1)
fid1 = calculate_frechet_distance(mu_orig1, sigma_orig1, mu_gen1, sigma_gen1)
print(fid1)
dataset = TextDataset(dataframe=df_text, tokenizer=bert_tokenizer, max_len=320)
loader = DataLoader(dataset, batch_size=16, shuffle=False)
bert_acc = evaluate(bert_model, loader)

print(bert_acc)


2.3264456599742505
(0.9734, {1: 0.9734})


In [ ]:
df_text=pd.read_csv("/content/drive/MyDrive/planB/df_generated_bart_class1_3.csv")
generated_texts1 = df_text['text'].tolist()
generated_embeddings1 = get_embeddings(generated_texts1)
mu_gen1, sigma_gen1 = calculate_mean_covariance(generated_embeddings1)
fid1 = calculate_frechet_distance(mu_orig1, sigma_orig1, mu_gen1, sigma_gen1)
print(fid1)

2.2600258330227123


In [ ]:
df_text=pd.read_csv("/content/drive/MyDrive/planB/df_generated_bart_class1_4.csv")
generated_texts1 = df_text['text'].tolist()
generated_embeddings1 = get_embeddings(generated_texts1)
mu_gen1, sigma_gen1 = calculate_mean_covariance(generated_embeddings1)
fid1 = calculate_frechet_distance(mu_orig1, sigma_orig1, mu_gen1, sigma_gen1)
print(fid1)

2.818059932058354


In [ ]:
df_text2=pd.read_csv("/content/drive/MyDrive/planB/df_generated_bart_class1_2.csv")
dataset = TextDataset(dataframe=df_text2, tokenizer=bert_tokenizer, max_len=320)
loader = DataLoader(dataset, batch_size=16, shuffle=False)
bert_acc = evaluate(bert_model, loader)
js = calculate_self_jaccard_similarity(df_text2)
print(bert_acc)
print(js)

(0.9761, {1: 0.9761})
{1: 0.07091136734796893}


In [ ]:
df_text3=pd.read_csv("/content/drive/MyDrive/planB/df_generated_bart_class1_3.csv")
dataset = TextDataset(dataframe=df_text3, tokenizer=bert_tokenizer, max_len=320)
loader = DataLoader(dataset, batch_size=16, shuffle=False)
bert_acc = evaluate(bert_model, loader)
js = calculate_self_jaccard_similarity(df_text3)
print(bert_acc)
print(js)

KeyboardInterrupt: 

In [ ]:
print(bert_acc)

(0.9729, {1: 0.9729})


In [ ]:
df_text3=pd.read_csv("/content/drive/MyDrive/planB/df_generated_bart_class0_2.csv")
dataset = TextDataset(dataframe=df_text3, tokenizer=bert_tokenizer, max_len=320)
loader = DataLoader(dataset, batch_size=16, shuffle=False)
bert_acc = evaluate(bert_model, loader)
# js = calculate_self_jaccard_similarity(df_text3)
print(bert_acc)
# print(js)

(0.9213, {0: 0.9213})


In [ ]:
df_text=pd.read_csv("/content/drive/MyDrive/planB/df_generated_bart_class0_2.csv")
generated_texts1 = df_text['text'].tolist()
generated_embeddings1 = get_embeddings(generated_texts1)
mu_gen1, sigma_gen1 = calculate_mean_covariance(generated_embeddings1)
fid1 = calculate_frechet_distance(mu_orig0, sigma_orig0, mu_gen1, sigma_gen1)
print(fid1)

1.5251623554021747


## load model 2

In [ ]:
from transformers import BartTokenizer, BartForConditionalGeneration
tokenizer = BartTokenizer.from_pretrained("/content/drive/MyDrive/model_723_bart_class1_epoch_29")
model = BartForConditionalGeneration.from_pretrained("/content/drive/MyDrive/model_723_bart_class1_epoch_29")

In [ ]:
import pandas as pd
import torch
device = 'cuda' if torch.cuda.is_available() else 'cpu'
model.eval()
model.to(device)

BartForConditionalGeneration(
  (model): BartModel(
    (shared): Embedding(50265, 768, padding_idx=1)
    (encoder): BartEncoder(
      (embed_tokens): BartScaledWordEmbedding(50265, 768, padding_idx=1)
      (embed_positions): BartLearnedPositionalEmbedding(1026, 768)
      (layers): ModuleList(
        (0-5): 6 x BartEncoderLayer(
          (self_attn): BartSdpaAttention(
            (k_proj): Linear(in_features=768, out_features=768, bias=True)
            (v_proj): Linear(in_features=768, out_features=768, bias=True)
            (q_proj): Linear(in_features=768, out_features=768, bias=True)
            (out_proj): Linear(in_features=768, out_features=768, bias=True)
          )
          (self_attn_layer_norm): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
          (activation_fn): GELUActivation()
          (fc1): Linear(in_features=768, out_features=3072, bias=True)
          (fc2): Linear(in_features=3072, out_features=768, bias=True)
          (final_layer_norm): Laye

In [ ]:
import torch
import pandas as pd
import random
import numpy as np

def set_seeds(seed):
    torch.manual_seed(seed)
    random.seed(seed)
    np.random.seed(seed)

def generate_text_with_beams(prompt, num_texts=5, temperature=1.0, top_k=50, top_p=0.95, min_length=50, max_length=320, num_beams=5, no_repeat_ngram_size=2, early_stopping=True, seed=42, repetition_penalty=1.5):
    if seed is not None:
        set_seeds(seed)
    model.eval()
    generated_texts = []
    for _ in range(num_texts):
        # Encode the prompt to generate input_ids and attention_mask
        encoding = tokenizer(prompt, return_tensors='pt', padding='max_length', max_length=max_length, truncation=True)
        input_ids = encoding['input_ids'].to(model.device)
        attention_mask = encoding['attention_mask'].to(model.device)

        outputs = model.generate(
            input_ids,
            attention_mask=attention_mask,  # Include attention mask in generation
            min_length=min_length,
            max_length=max_length,
            num_beams=num_beams,
            num_return_sequences=1,  # Generate one sequence at a time to ensure diversity
            temperature=temperature,
            top_k=top_k,
            top_p=top_p,
            no_repeat_ngram_size=no_repeat_ngram_size,
            repetition_penalty=repetition_penalty,
            do_sample=True,
            pad_token_id=tokenizer.eos_token_id,
            early_stopping=early_stopping
        )

        text = tokenizer.decode(outputs[0], skip_special_tokens=True)
        generated_texts.append(text)

    return generated_texts

def generate_text(top_p, top_k, temperature, num_beams=5, num_texts=20, seed=42, repetition_penalty=1.2):
    prompts = [ "Write a Science/Technology News"]
    all_prompts = []
    all_texts = []

    for prompt in prompts:
        generated_texts_for_prompt = []
        num_iterations = num_texts // num_beams

        for i in range(num_iterations):
            set_seeds(seed + i)
            generated_texts = generate_text_with_beams(
                prompt=prompt,
                num_texts=num_beams,
                temperature=temperature,
                top_k=top_k,
                top_p=top_p,
                min_length=50,
                max_length=200,  # Adjust max_length as needed
                num_beams=num_beams,
                no_repeat_ngram_size=2,
                early_stopping=True,  # Enable early stopping
                seed=seed + i,  # Adjust seed for each batch to maintain diversity while being reproducible
                repetition_penalty=repetition_penalty  # Adding repetition penalty
            )
            generated_texts_for_prompt.extend(generated_texts)

        # Ensure uniqueness and limit to the desired number of texts
        unique_texts = list(set(generated_texts_for_prompt))[:num_texts]
        all_prompts.extend([prompt] * len(unique_texts))
        all_texts.extend(unique_texts)

    return pd.DataFrame({'prompt': all_prompts, 'text': all_texts})


In [ ]:
import torch
from torch.utils.data import Dataset, DataLoader
class TextDataset(Dataset):
    def __init__(self, dataframe, tokenizer, max_len=320):
        self.tokenizer = tokenizer
        self.data = dataframe
        self.text = dataframe.text
        self.targets = dataframe.label
        self.max_len = max_len

    def __len__(self):
        return len(self.text)

    def __getitem__(self, index):
        text = str(self.text[index])
        text = " ".join(text.split())

        inputs = self.tokenizer.encode_plus(
            text,
            None,
            add_special_tokens=True,
            max_length=self.max_len,
            padding='max_length',
            truncation=True,
            return_attention_mask=True,
            return_tensors="pt",
        )

        return {
            'input_ids': inputs['input_ids'].flatten(),
            'attention_mask': inputs['attention_mask'].flatten(),
            'labels': torch.tensor(self.targets[index], dtype=torch.long)
        }


In [ ]:
from transformers import BertTokenizer, BertForSequenceClassification
import torch
device = 'cuda' if torch.cuda.is_available() else 'cpu'
from sklearn.metrics import accuracy_score
import numpy as np
from collections import defaultdict
from transformers import BertTokenizer
from transformers import BertForSequenceClassification
bert_tokenizer = BertTokenizer.from_pretrained('/content/drive/MyDrive/planB/bert/BERT_baseline_723_2')
bert_model = BertForSequenceClassification.from_pretrained('/content/drive/MyDrive/planB/bert/BERT_baseline_723_2', num_labels=2)
bert_model.to(device)
def evaluate(model, dataloader):
    model.eval()

    predictions, true_labels = [], []

    with torch.no_grad():
        for batch in dataloader:
            input_ids = batch['input_ids'].to(device)
            attention_mask = batch['attention_mask'].to(device)
            labels = batch['labels'].to(device)

            outputs = model(input_ids, attention_mask=attention_mask, labels=labels)

            logits = outputs.logits.detach().cpu().numpy()
            label_ids = labels.to('cpu').numpy()

            predictions.extend(np.argmax(logits, axis=1).flatten())
            true_labels.extend(label_ids.flatten())

    avg_accuracy = accuracy_score(true_labels, predictions)

    # Calculate accuracy for each class
    class_accuracies = defaultdict(list)
    for true, pred in zip(true_labels, predictions):
        class_accuracies[true].append(pred == true)

    class_accuracy_results = {label: np.mean(acc) for label, acc in class_accuracies.items()}

    return avg_accuracy, class_accuracy_results

In [ ]:
!pip install sacrebleu

import pandas as pd
import numpy as np
import sacrebleu

def calculate_self_bleu(texts):
    scores = []
    for i, target in enumerate(texts):
        references = texts[:i] + texts[i+1:]
        bleu_score = sacrebleu.corpus_bleu([target], [references]).score
        scores.append(bleu_score)
    return np.mean(scores)

def calculate_self_bleu_per_class(df):
    class_bleu_scores = {}
    for label, group in df.groupby('label'):
        texts = group['text'].tolist()
        self_bleu_score = calculate_self_bleu(texts)
        class_bleu_scores[label] = self_bleu_score
    return class_bleu_scores

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.0/58.0 kB 2.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 106.7/106.7 kB 4.8 MB/s eta 0:00:00


In [ ]:
import pandas as pd
from sklearn.metrics import jaccard_score
from itertools import combinations
import numpy as np

# Function to calculate Jaccard Similarity between two texts
def jaccard_similarity(text1, text2):
    set1 = set(text1.split())
    set2 = set(text2.split())
    intersection = set1.intersection(set2)
    union = set1.union(set2)
    return len(intersection) / len(union) if len(union) > 0 else 0

# Function to calculate mean Jaccard similarity within each class
def calculate_self_jaccard_similarity(df):
    results = {}
    for label, group in df.groupby('label'):
        jaccard_similarities = [jaccard_similarity(text1, text2) for text1, text2 in combinations(group['text'], 2)]
        results[label] = np.mean(jaccard_similarities) if jaccard_similarities else 0
    return results

In [ ]:
label_map2 = {"Write a Business News":0, "Write a Science/Technology News":1}

In [ ]:
import numpy as np
import torch
from transformers import BertTokenizer, BertModel
from scipy.linalg import sqrtm
import pandas as pd

# Load BERT model and tokenizer
bert_tokenizer_base = BertTokenizer.from_pretrained('bert-base-uncased')
bert_model_base = BertModel.from_pretrained('bert-base-uncased')
bert_model_base.to(device)
def get_embeddings(texts, batch_size=32):
    all_embeddings = []
    for i in range(0, len(texts), batch_size):
        batch_texts = texts[i:i+batch_size]
        inputs = bert_tokenizer_base(batch_texts, return_tensors='pt', padding=True, truncation=True, max_length=320)
        inputs = {k: v.to(device) for k, v in inputs.items()}
        with torch.no_grad():
            outputs = bert_model_base(**inputs)
        embeddings = outputs.last_hidden_state.mean(dim=1)
        all_embeddings.append(embeddings.cpu().numpy())
        torch.cuda.empty_cache()  # Clear unused memory
    return np.vstack(all_embeddings)

# Function to calculate mean and covariance
def calculate_mean_covariance(embeddings):
    mu = np.mean(embeddings, axis=0)
    sigma = np.cov(embeddings, rowvar=False)
    return mu, sigma

# Function to compute Fréchet Distance
def calculate_frechet_distance(mu1, sigma1, mu2, sigma2):
    diff = mu1 - mu2
    covmean, _ = sqrtm(sigma1.dot(sigma2), disp=False)
    if np.iscomplexobj(covmean):
        covmean = covmean.real
    return np.sum(diff**2) + np.trace(sigma1 + sigma2 - 2*covmean)

tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

config.json:   0%|          | 0.00/570 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/440M [00:00<?, ?B/s]

In [ ]:
df_original0 = df_train[df_train['label']==0].sample(n=500, random_state=13)
original_texts0 = df_original0['text'].tolist()
original_embeddings0 = get_embeddings(original_texts0)
df_original1 = df_train[df_train['label']==1].sample(n=500, random_state=13)
original_texts1 = df_original1['text'].tolist()
original_embeddings1 = get_embeddings(original_texts1)
mu_orig0, sigma_orig0 = calculate_mean_covariance(original_embeddings0)
mu_orig1, sigma_orig1 = calculate_mean_covariance(original_embeddings1)

In [ ]:
for t in [1.3,1.5,1.7,1.9]:
  df_text = generate_text(top_p=0.95, top_k=45, temperature=t, num_beams=4 , num_texts=500, repetition_penalty=1.7,seed=724)
  df_text['label'] = df_text['prompt'].map(label_map2)
  dataset = TextDataset(dataframe=df_text, tokenizer=bert_tokenizer, max_len=320)
  loader = DataLoader(dataset, batch_size=16, shuffle=False)
  bert_acc = evaluate(bert_model, loader)
  js = calculate_self_jaccard_similarity(df_text)
  sb = calculate_self_bleu_per_class(df_text)
  df_generate1 = df_text[df_text['label']==1]
  generated_texts1 = df_generate1['text'].tolist()
  generated_embeddings1 = get_embeddings(generated_texts1)
  mu_gen1, sigma_gen1 = calculate_mean_covariance(generated_embeddings1)

  fid1 = calculate_frechet_distance(mu_orig1, sigma_orig1, mu_gen1, sigma_gen1)
  print(f"t: {t}")
  print(f"bert accuracy: {bert_acc}")
  print(f"Jaccard similarities: {js}")
  print(f"Self-BLEU: {sb}")
  print(fid1)

t: 1.3
bert accuracy: (0.5235235235235235, {0: 0.096, 1: 0.9519038076152304})
Jaccard similarities: {0: 0.0870428760938569, 1: 0.08918436004448158}
Self-BLEU: {0: 2.5848432761452704, 1: 3.8890658419179096}
10.2625502824106
t: 1.5
bert accuracy: (0.527, {0: 0.086, 1: 0.968})
Jaccard similarities: {0: 0.0871861712305975, 1: 0.0871673449568837}
Self-BLEU: {0: 2.210022240370516, 1: 2.179241245277493}
7.953333606089343
t: 1.7
bert accuracy: (0.51, {0: 0.048, 1: 0.972})
Jaccard similarities: {0: 0.08734991385116816, 1: 0.08509917597381911}
Self-BLEU: {0: 2.3674546869335504, 1: 2.3430414437151494}
7.832641944434352
t: 1.9
bert accuracy: (0.496, {0: 0.036, 1: 0.956})
Jaccard similarities: {0: 0.0833946824675723, 1: 0.08243804312171027}
Self-BLEU: {0: 1.2384418495637588, 1: 1.429856907100955}
7.610371144073297


In [ ]:
for t in [2.3,2.5,2.7,2.9,3.1]:
  df_text = generate_text(top_p=0.95, top_k=45, temperature=t, num_beams=4 , num_texts=500, repetition_penalty=1.7,seed=724)
  df_text['label'] = df_text['prompt'].map(label_map2)
  dataset = TextDataset(dataframe=df_text, tokenizer=bert_tokenizer, max_len=320)
  loader = DataLoader(dataset, batch_size=16, shuffle=False)
  bert_acc = evaluate(bert_model, loader)
  js = calculate_self_jaccard_similarity(df_text)
  sb = calculate_self_bleu_per_class(df_text)
  df_generate1 = df_text[df_text['label']==1]
  generated_texts1 = df_generate1['text'].tolist()
  generated_embeddings1 = get_embeddings(generated_texts1)
  mu_gen1, sigma_gen1 = calculate_mean_covariance(generated_embeddings1)

  fid1 = calculate_frechet_distance(mu_orig1, sigma_orig1, mu_gen1, sigma_gen1)
  print(f"t: {t}")
  print(f"bert accuracy: {bert_acc}")
  print(f"Jaccard similarities: {js}")
  print(f"Self-BLEU: {sb}")
  print(fid1)

t: 2.3
bert accuracy: (0.968, {1: 0.968})
Jaccard similarities: {1: 0.07856889008398389}
Self-BLEU: {1: 2.2337209393695687}
6.470097626178939
t: 2.5
bert accuracy: (0.964, {1: 0.964})
Jaccard similarities: {1: 0.07703429900280309}
Self-BLEU: {1: 1.823258597857071}
6.115016316086627
t: 2.7
bert accuracy: (0.988, {1: 0.988})
Jaccard similarities: {1: 0.07674379726623658}
Self-BLEU: {1: 1.3668650412744474}
5.5990748459883575
t: 2.9
bert accuracy: (0.978, {1: 0.978})
Jaccard similarities: {1: 0.07657324046299654}
Self-BLEU: {1: 1.4529860283786133}
5.109976781355443
t: 3.1
bert accuracy: (0.964, {1: 0.964})
Jaccard similarities: {1: 0.07446818667907666}
Self-BLEU: {1: 1.9603892499531566}
4.921762779387917


In [ ]:
for t in [3.3,3.7]:
  df_text = generate_text(top_p=0.95, top_k=45, temperature=t, num_beams=4 , num_texts=500, repetition_penalty=1.7,seed=724)
  df_text['label'] = df_text['prompt'].map(label_map2)
  dataset = TextDataset(dataframe=df_text, tokenizer=bert_tokenizer, max_len=320)
  loader = DataLoader(dataset, batch_size=16, shuffle=False)
  bert_acc = evaluate(bert_model, loader)
  js = calculate_self_jaccard_similarity(df_text)
  sb = calculate_self_bleu_per_class(df_text)
  df_generate1 = df_text[df_text['label']==1]
  generated_texts1 = df_generate1['text'].tolist()
  generated_embeddings1 = get_embeddings(generated_texts1)
  mu_gen1, sigma_gen1 = calculate_mean_covariance(generated_embeddings1)

  fid1 = calculate_frechet_distance(mu_orig1, sigma_orig1, mu_gen1, sigma_gen1)
  print(f"t: {t}")
  print(f"bert accuracy: {bert_acc}")
  print(f"Jaccard similarities: {js}")
  print(f"Self-BLEU: {sb}")
  print(fid1)

t: 3.3
bert accuracy: (0.978, {1: 0.978})
Jaccard similarities: {1: 0.07415329918789852}
Self-BLEU: {1: 1.3921928858021606}
4.649758382979603
t: 3.7
bert accuracy: (0.97, {1: 0.97})
Jaccard similarities: {1: 0.06986815749740463}
Self-BLEU: {1: 0.915999204020242}
4.173382348133012


In [ ]:
df_text[0:2].text.to_list()

['Fossil finds clues of dinosaur DNA fossil AP AP - A fossil described by the scientists who unearthed it has yielded hints of an extinct human origin, suggesting that dinosaurs may have been common to early human sexual history.What could prove? Researchers say they have found a fossil of some sort that suggests that possibly contains dinosaur bones dating back to pre-Genetic era times and suggest that the discovery is very likely to lead to ',
 'New media tools should make the DVD players obsolete Apple Computer is making its iPod more portable--but it 39s going to take some time. By Kim Kasser. MacKasser and Paul Goulston of Phony Ink Designs have made their mark.']

In [ ]:
for k in [40,45,50,55]:
  df_text = generate_text(top_p=0.95, top_k=k, temperature=4.7, num_beams=4 , num_texts=500, repetition_penalty=1.7,seed=724)
  df_text['label'] = df_text['prompt'].map(label_map2)
  dataset = TextDataset(dataframe=df_text, tokenizer=bert_tokenizer, max_len=320)
  loader = DataLoader(dataset, batch_size=16, shuffle=False)
  bert_acc = evaluate(bert_model, loader)
  js = calculate_self_jaccard_similarity(df_text)
  sb = calculate_self_bleu_per_class(df_text)
  df_generate1 = df_text[df_text['label']==1]
  generated_texts1 = df_generate1['text'].tolist()
  generated_embeddings1 = get_embeddings(generated_texts1)
  mu_gen1, sigma_gen1 = calculate_mean_covariance(generated_embeddings1)

  fid1 = calculate_frechet_distance(mu_orig1, sigma_orig1, mu_gen1, sigma_gen1)
  print(f"k: {k}")
  print(f"bert accuracy: {bert_acc}")
  print(f"Jaccard similarities: {js}")
  print(f"Self-BLEU: {sb}")
  print(fid1)
  ## this one

k: 40
bert accuracy: (0.964, {1: 0.964})
Jaccard similarities: {1: 0.0682727366999546}
Self-BLEU: {1: 1.1570350197425474}
4.166705736035113
k: 45
bert accuracy: (0.978, {1: 0.978})
Jaccard similarities: {1: 0.06839764380336283}
Self-BLEU: {1: 0.7580151667208217}
4.189978223949332
k: 50
bert accuracy: (0.972, {1: 0.972})
Jaccard similarities: {1: 0.0656722755049557}
Self-BLEU: {1: 1.1663597529016727}
4.0101031876121525
k: 55
bert accuracy: (0.982, {1: 0.982})
Jaccard similarities: {1: 0.0671324449138463}
Self-BLEU: {1: 1.0160026353780716}
4.208755718388127


In [ ]:
for rp in [1,1.3,2.1,2.5]:
  df_text = generate_text(top_p=0.95, top_k=50, temperature=4.7, num_beams=4 , num_texts=500, repetition_penalty=rp,seed=724)
  df_text['label'] = df_text['prompt'].map(label_map2)
  dataset = TextDataset(dataframe=df_text, tokenizer=bert_tokenizer, max_len=320)
  loader = DataLoader(dataset, batch_size=16, shuffle=False)
  bert_acc = evaluate(bert_model, loader)
  js = calculate_self_jaccard_similarity(df_text)
  sb = calculate_self_bleu_per_class(df_text)
  df_generate1 = df_text[df_text['label']==1]
  generated_texts1 = df_generate1['text'].tolist()
  generated_embeddings1 = get_embeddings(generated_texts1)
  mu_gen1, sigma_gen1 = calculate_mean_covariance(generated_embeddings1)

  fid1 = calculate_frechet_distance(mu_orig1, sigma_orig1, mu_gen1, sigma_gen1)
  print(f"rp: {rp}")
  print(f"bert accuracy: {bert_acc}")
  print(f"Jaccard similarities: {js}")
  print(f"Self-BLEU: {sb}")
  print(fid1)
  ## this one

rp: 1
bert accuracy: (0.966, {1: 0.966})
Jaccard similarities: {1: 0.06118992783874104}
Self-BLEU: {1: 1.1189134341952551}
4.056347966117926
rp: 1.3
bert accuracy: (0.97, {1: 0.97})
Jaccard similarities: {1: 0.06542907639020785}
Self-BLEU: {1: 0.9900671090866908}
4.067178225486829
rp: 2.1
bert accuracy: (0.974, {1: 0.974})
Jaccard similarities: {1: 0.06836704852548661}
Self-BLEU: {1: 1.261080344945408}
4.230261224738015
rp: 2.5
bert accuracy: (0.976, {1: 0.976})
Jaccard similarities: {1: 0.06711456412539314}
Self-BLEU: {1: 1.0941211899525527}
4.214707249644949


In [ ]:
df_text[0:2].text.to_list()

['Nasa spacecraft to strike strike Saturn Tonight NASA will enter space with its first unmanned craft of the future, after a successful corvette launched into the international space port at 1102 AM Pacific time Wednesday. Space craft would be the first humans to do so ',
 'FCC Cuts Price On Computer Software The European Computer Association CPDA has dropped its price on computer software - a move that could encourage manufacturers to add the software products, but is likely to draw regulatory opposition for such products. PLC will provide a link for its Web site. ltFONT faceverdana']

In [ ]:
df_text = generate_text(top_p=0.95, top_k=45, temperature=3.1, num_beams=4 , num_texts=10000, repetition_penalty=1.7,seed=729)
df_text['label'] = df_text['prompt'].map(label_map2)
df_text.to_csv("/content/drive/MyDrive/planB/df_generated_bart_class1_0.csv")

In [ ]:
df_text=pd.read_csv("/content/drive/MyDrive/planB/df_generated_bart_class1_0.csv")

In [ ]:
dataset = TextDataset(dataframe=df_text, tokenizer=bert_tokenizer, max_len=320)
loader = DataLoader(dataset, batch_size=16, shuffle=False)
bert_acc = evaluate(bert_model, loader)
js = calculate_self_jaccard_similarity(df_text)
print(bert_acc)
print(js)

(0.9768, {1: 0.9768})
{1: 0.07442262581479629}


In [ ]:
df_text1=pd.read_csv("/content/drive/MyDrive/planB/df_generated_bart_class1_1.csv")
dataset = TextDataset(dataframe=df_text1, tokenizer=bert_tokenizer, max_len=320)
loader = DataLoader(dataset, batch_size=16, shuffle=False)
bert_acc = evaluate(bert_model, loader)
js = calculate_self_jaccard_similarity(df_text1)
print(bert_acc)
print(js)

(0.9779, {1: 0.9779})
{1: 0.07319351211725297}


In [ ]:
df_text2=pd.read_csv("/content/drive/MyDrive/planB/df_generated_bart_class1_2.csv")
dataset = TextDataset(dataframe=df_text1, tokenizer=bert_tokenizer, max_len=320)
loader = DataLoader(dataset, batch_size=16, shuffle=False)
bert_acc = evaluate(bert_model, loader)
js = calculate_self_jaccard_similarity(df_text2)
print(bert_acc)
print(js)

(0.9779, {1: 0.9779})
{1: 0.07091136734796893}


In [ ]:
df_original1 = df_train[df_train['label']==1][0:1000]
original_texts1 = df_original1['text'].tolist()
original_embeddings1 = get_embeddings(original_texts1)
mu_orig1, sigma_orig1 = calculate_mean_covariance(original_embeddings1)
df_generate1 = df_train[df_train['label']==1][1000:2000]
generated_texts1 = df_generate1['text'].tolist()
generated_embeddings1 = get_embeddings(generated_texts1)
mu_gen1, sigma_gen1 = calculate_mean_covariance(generated_embeddings1)
fid1 = calculate_frechet_distance(mu_orig1, sigma_orig1, mu_gen1, sigma_gen1)
print(fid1)

1.8417605206148489


In [ ]:
df_original1 = df_train[df_train['label']==1][0:1000]
original_texts1 = df_original1['text'].tolist()
original_embeddings1 = get_embeddings(original_texts1)
mu_orig1, sigma_orig1 = calculate_mean_covariance(original_embeddings1)
df_generate1 = df_train[df_train['label']==0][1000:2000]
generated_texts1 = df_generate1['text'].tolist()
generated_embeddings1 = get_embeddings(generated_texts1)
mu_gen1, sigma_gen1 = calculate_mean_covariance(generated_embeddings1)
fid1 = calculate_frechet_distance(mu_orig1, sigma_orig1, mu_gen1, sigma_gen1)
print(fid1)

8.183885243008145


In [ ]:
df_text=pd.read_csv("/content/drive/MyDrive/planB/df_generated_bart_class1_0.csv")
generated_texts1 = df_text['text'].tolist()
generated_embeddings1 = get_embeddings(generated_texts1)
mu_gen1, sigma_gen1 = calculate_mean_covariance(generated_embeddings1)
fid1 = calculate_frechet_distance(mu_orig1, sigma_orig1, mu_gen1, sigma_gen1)
print(fid1)

3.21268893045666


In [ ]:
df_text=pd.read_csv("/content/drive/MyDrive/planB/df_generated_bart_class1_1.csv")
generated_texts1 = df_text['text'].tolist()
generated_embeddings1 = get_embeddings(generated_texts1)
mu_gen1, sigma_gen1 = calculate_mean_covariance(generated_embeddings1)
fid1 = calculate_frechet_distance(mu_orig1, sigma_orig1, mu_gen1, sigma_gen1)
print(fid1)

3.0926786144048206


In [ ]:
df_text=pd.read_csv("/content/drive/MyDrive/planB/df_generated_bart_class1_2.csv")
generated_texts1 = df_text['text'].tolist()
generated_embeddings1 = get_embeddings(generated_texts1)
mu_gen1, sigma_gen1 = calculate_mean_covariance(generated_embeddings1)
fid1 = calculate_frechet_distance(mu_orig1, sigma_orig1, mu_gen1, sigma_gen1)
print(fid1)

2.782454053963


In [ ]:
df_text

,Unnamed: 0,prompt,text,label
0,0,Write a Science/Technology News,"Oracle, SBC reach Deal to Buy Oracle Business ...",1
1,1,Write a Science/Technology News,"Yahoo Offers Online Music Search, PayPal Reute...",1
2,2,Write a Science/Technology News,Toshiba to unveil notebook memory unit Toshiba...,1
3,3,Write a Science/Technology News,Nvidia Introduces Powerful 3G G5 GPU Nvidia Co...,1
4,4,Write a Science/Technology News,AOL Offers Web-Swap Service Microsoft has intr...,1
...,...,...,...,...
9995,9995,Write a Science/Technology News,NASA 39s Titan Probe lands on Mars NASA offici...,1
9996,9996,Write a Science/Technology News,Microsoft updates Windows XP update Microsoft ...,1
9997,9997,Write a Science/Technology News,Intel Adds Wireless Chips Intel announced that...,1
9998,9998,Write a Science/Technology News,Nvidia To Build New GeForce X800 GPU Nvidia to...,1


In [ ]:
df_text = generate_text(top_p=0.95, top_k=45, temperature=3.3, num_beams=4 , num_texts=10000, repetition_penalty=1.7,seed=729)
df_text['label'] = df_text['prompt'].map(label_map2)
df_text.to_csv("/content/drive/MyDrive/planB/df_generated_bart_class1_1.csv")

In [ ]:
df_text = generate_text(top_p=0.95, top_k=45, temperature=3.7, num_beams=4 , num_texts=10000, repetition_penalty=1.7,seed=729)
df_text['label'] = df_text['prompt'].map(label_map2)
df_text.to_csv("/content/drive/MyDrive/planB/df_generated_bart_class1_2.csv")

In [ ]:
df_text = generate_text(top_p=0.95, top_k=45, temperature=4.1, num_beams=4 , num_texts=10000, repetition_penalty=1.7,seed=729)
df_text['label'] = df_text['prompt'].map(label_map2)
df_text.to_csv("/content/drive/MyDrive/planB/df_generated_bart_class1_3.csv")

In [ ]:
df_text = generate_text(top_p=0.95, top_k=45, temperature=4.7, num_beams=4 , num_texts=10000, repetition_penalty=1.7,seed=729)
df_text['label'] = df_text['prompt'].map(label_map2)
df_text.to_csv("/content/drive/MyDrive/planB/df_generated_bart_class1_4.csv")

In [ ]:
df_text = generate_text(top_p=0.95, top_k=50, temperature=4.7, num_beams=4 , num_texts=10000, repetition_penalty=1.7,seed=729)
df_text['label'] = df_text['prompt'].map(label_map2)
df_text.to_csv("/content/drive/MyDrive/planB/df_generated_bart_class1_5.csv")

In [ ]:
df_original0 = df_train[df_train['label']==0]
original_texts0 = df_original0['text'].tolist()
original_embeddings0 = get_embeddings(original_texts0)
mu_orig0, sigma_orig0 = calculate_mean_covariance(original_embeddings0)
df_original1 = df_train[df_train['label']==1]
original_texts1 = df_original1['text'].tolist()
original_embeddings1 = get_embeddings(original_texts1)
mu_orig1, sigma_orig1 = calculate_mean_covariance(original_embeddings1)

In [ ]:
df_text=pd.read_csv("/content/drive/MyDrive/planB/df_generated_bart_class1_5.csv")
generated_texts1 = df_text['text'].tolist()
generated_embeddings1 = get_embeddings(generated_texts1)
mu_gen1, sigma_gen1 = calculate_mean_covariance(generated_embeddings1)
fid1 = calculate_frechet_distance(mu_orig1, sigma_orig1, mu_gen1, sigma_gen1)
print(fid1)
dataset = TextDataset(dataframe=df_text, tokenizer=bert_tokenizer, max_len=320)
loader = DataLoader(dataset, batch_size=16, shuffle=False)
bert_acc = evaluate(bert_model, loader)

print(bert_acc)


2.3264456599742505
(0.9734, {1: 0.9734})


In [ ]:
df_text=pd.read_csv("/content/drive/MyDrive/planB/df_generated_bart_class1_3.csv")
generated_texts1 = df_text['text'].tolist()
generated_embeddings1 = get_embeddings(generated_texts1)
mu_gen1, sigma_gen1 = calculate_mean_covariance(generated_embeddings1)
fid1 = calculate_frechet_distance(mu_orig1, sigma_orig1, mu_gen1, sigma_gen1)
print(fid1)

2.2600258330227123


In [ ]:
df_text=pd.read_csv("/content/drive/MyDrive/planB/df_generated_bart_class1_4.csv")
generated_texts1 = df_text['text'].tolist()
generated_embeddings1 = get_embeddings(generated_texts1)
mu_gen1, sigma_gen1 = calculate_mean_covariance(generated_embeddings1)
fid1 = calculate_frechet_distance(mu_orig1, sigma_orig1, mu_gen1, sigma_gen1)
print(fid1)

2.818059932058354


In [ ]:
df_text2=pd.read_csv("/content/drive/MyDrive/planB/df_generated_bart_class1_2.csv")
dataset = TextDataset(dataframe=df_text2, tokenizer=bert_tokenizer, max_len=320)
loader = DataLoader(dataset, batch_size=16, shuffle=False)
bert_acc = evaluate(bert_model, loader)
js = calculate_self_jaccard_similarity(df_text2)
print(bert_acc)
print(js)

(0.9761, {1: 0.9761})
{1: 0.07091136734796893}


In [ ]:
df_text3=pd.read_csv("/content/drive/MyDrive/planB/df_generated_bart_class1_3.csv")
dataset = TextDataset(dataframe=df_text3, tokenizer=bert_tokenizer, max_len=320)
loader = DataLoader(dataset, batch_size=16, shuffle=False)
bert_acc = evaluate(bert_model, loader)
js = calculate_self_jaccard_similarity(df_text3)
print(bert_acc)
print(js)

KeyboardInterrupt: 

In [ ]:
print(bert_acc)

(0.9729, {1: 0.9729})


In [ ]:
df_text3=pd.read_csv("/content/drive/MyDrive/planB/df_generated_bart_class0_2.csv")
dataset = TextDataset(dataframe=df_text3, tokenizer=bert_tokenizer, max_len=320)
loader = DataLoader(dataset, batch_size=16, shuffle=False)
bert_acc = evaluate(bert_model, loader)
# js = calculate_self_jaccard_similarity(df_text3)
print(bert_acc)
# print(js)

(0.9213, {0: 0.9213})


In [ ]:
df_text=pd.read_csv("/content/drive/MyDrive/planB/df_generated_bart_class0_2.csv")
generated_texts1 = df_text['text'].tolist()
generated_embeddings1 = get_embeddings(generated_texts1)
mu_gen1, sigma_gen1 = calculate_mean_covariance(generated_embeddings1)
fid1 = calculate_frechet_distance(mu_orig0, sigma_orig0, mu_gen1, sigma_gen1)
print(fid1)

1.5251623554021747


## other

In [ ]:
df_text1 = generate_text(top_p=0.95, top_k=50, temperature=1.2, num_beams=4 , num_texts=10000, repetition_penalty=1.7,seed=724)
df_text1['label'] = df_text1['prompt'].map(label_map2)
df_text1.to_csv("/content/drive/MyDrive/planB/df_generated_bartall_0.csv")

In [ ]:
df_text2 = generate_text(top_p=0.95, top_k=55, temperature=1.4, num_beams=4 , num_texts=10005, repetition_penalty=1.7,seed=724)
df_text2['label'] = df_text2['prompt'].map(label_map2)
df_text2.to_csv("/content/drive/MyDrive/planB/df_generated_bartall_1.csv")

NameError: name 'generate_text' is not defined

In [ ]:
dataset = TextDataset(dataframe=df_text, tokenizer=bert_tokenizer, max_len=320)
loader = DataLoader(dataset, batch_size=16, shuffle=False)
bert_acc = evaluate(bert_model, loader)
js = calculate_self_jaccard_similarity(df_text)


In [ ]:
import numpy as np
import torch
from transformers import BertTokenizer, BertModel
from scipy.linalg import sqrtm
import pandas as pd

# Load BERT model and tokenizer
bert_tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')
bert_model = BertModel.from_pretrained('bert-base-uncased')

# Function to get embeddings
def get_embeddings(texts):
    inputs = bert_tokenizer(texts, return_tensors='pt', padding=True, truncation=True, max_length=512)
    with torch.no_grad():
        outputs = bert_model(**inputs)
    embeddings = outputs.last_hidden_state.mean(dim=1)  # Use mean of token embeddings
    return embeddings.numpy()

# Function to calculate mean and covariance
def calculate_mean_covariance(embeddings):
    mu = np.mean(embeddings, axis=0)
    sigma = np.cov(embeddings, rowvar=False)
    return mu, sigma

# Function to compute Fréchet Distance
def calculate_frechet_distance(mu1, sigma1, mu2, sigma2):
    diff = mu1 - mu2
    covmean, _ = sqrtm(sigma1.dot(sigma2), disp=False)
    if np.iscomplexobj(covmean):
        covmean = covmean.real
    return np.sum(diff**2) + np.trace(sigma1 + sigma2 - 2*covmean)


In [ ]:
df_original = df_train[df_train['label']==2]
df_generate = df_text0[df_text0['label']==0]
original_texts = df_original['text'].tolist()
generated_texts = df_generate['text'].tolist()
original_embeddings = get_embeddings(original_texts)
generated_embeddings = get_embeddings(generated_texts)

# Calculate mean and covariance
mu_orig, sigma_orig = calculate_mean_covariance(original_embeddings)
mu_gen, sigma_gen = calculate_mean_covariance(generated_embeddings)

# Compute Fréchet Distance
fid = calculate_frechet_distance(mu_orig, sigma_orig, mu_gen, sigma_gen)
print("Fréchet Distance:", fid)

Fréchet Distance: 15.71881764392105


In [ ]:
df_original = df_train[df_train['label']==3]
df_generate = df_text0[df_text0['label']==1]
original_texts = df_original['text'].tolist()
generated_texts = df_generate['text'].tolist()
original_embeddings = get_embeddings(original_texts)
generated_embeddings = get_embeddings(generated_texts)

# Calculate mean and covariance
mu_orig, sigma_orig = calculate_mean_covariance(original_embeddings)
mu_gen, sigma_gen = calculate_mean_covariance(generated_embeddings)

# Compute Fréchet Distance
fid = calculate_frechet_distance(mu_orig, sigma_orig, mu_gen, sigma_gen)
print("Fréchet Distance:", fid)

Fréchet Distance: 17.05582525451667


In [ ]:
df_original = df_train[df_train['label']==2]
df_generate = df_text1[df_text1['label']==0]
original_texts = df_original['text'].tolist()
generated_texts = df_generate['text'].tolist()
original_embeddings = get_embeddings(original_texts)
generated_embeddings = get_embeddings(generated_texts)

# Calculate mean and covariance
mu_orig, sigma_orig = calculate_mean_covariance(original_embeddings)
mu_gen, sigma_gen = calculate_mean_covariance(generated_embeddings)

# Compute Fréchet Distance
fid = calculate_frechet_distance(mu_orig, sigma_orig, mu_gen, sigma_gen)
print("Fréchet Distance:", fid)

Fréchet Distance: 16.053433885655654


In [ ]:
df_original = df_train[df_train['label']==3]
df_generate = df_text1[df_text1['label']==1]
original_texts = df_original['text'].tolist()
generated_texts = df_generate['text'].tolist()
original_embeddings = get_embeddings(original_texts)
generated_embeddings = get_embeddings(generated_texts)

# Calculate mean and covariance
mu_orig, sigma_orig = calculate_mean_covariance(original_embeddings)
mu_gen, sigma_gen = calculate_mean_covariance(generated_embeddings)

# Compute Fréchet Distance
fid = calculate_frechet_distance(mu_orig, sigma_orig, mu_gen, sigma_gen)
print("Fréchet Distance:", fid)

Fréchet Distance: 16.599486418833028


In [ ]:
df_text1=pd.read_csv("/content/drive/MyDrive/planB/df_generated_bartall_1.csv")

In [ ]:
df_original = df_train[df_train['label']==2]
df_generate = df_text1[df_text1['label']==0]
original_texts = df_original['text'].tolist()
generated_texts = df_generate['text'].tolist()
original_embeddings = get_embeddings(original_texts)
generated_embeddings = get_embeddings(generated_texts)

# Calculate mean and covariance
mu_orig, sigma_orig = calculate_mean_covariance(original_embeddings)
mu_gen, sigma_gen = calculate_mean_covariance(generated_embeddings)

# Compute Fréchet Distance
fid = calculate_frechet_distance(mu_orig, sigma_orig, mu_gen, sigma_gen)
print("Fréchet Distance:", fid)

Fréchet Distance: 12.20276166015872


In [ ]:
df_original = df_train[df_train['label']==2]
df_generate = test_set[test_set['label']==2]
original_texts = df_original['text'].tolist()
generated_texts = df_generate['text'].tolist()
original_embeddings = get_embeddings(original_texts)
generated_embeddings = get_embeddings(generated_texts)

# Calculate mean and covariance
mu_orig, sigma_orig = calculate_mean_covariance(original_embeddings)
mu_gen, sigma_gen = calculate_mean_covariance(generated_embeddings)

# Compute Fréchet Distance
fid = calculate_frechet_distance(mu_orig, sigma_orig, mu_gen, sigma_gen)
print("Fréchet Distance:", fid)

Fréchet Distance: 1.3579296077970753


In [ ]:
!pip install sacrebleu

import pandas as pd
import numpy as np
import sacrebleu

def calculate_self_bleu(texts):
    scores = []
    for i, target in enumerate(texts):
        references = texts[:i] + texts[i+1:]
        bleu_score = sacrebleu.corpus_bleu([target], [references]).score
        scores.append(bleu_score)
    return np.mean(scores)

def calculate_self_bleu_per_class(df):
    class_bleu_scores = {}
    for label, group in df.groupby('label'):
        texts = group['text'].tolist()
        self_bleu_score = calculate_self_bleu(texts)
        class_bleu_scores[label] = self_bleu_score
    return class_bleu_scores

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 106.7/106.7 kB 3.4 MB/s eta 0:00:00


In [ ]:
print(calculate_self_bleu_per_class(df_text1))

KeyboardInterrupt: 

In [ ]:
text_1_0=df_text1[df_text1["label"]==0].sample(n=1000, random_state=7)
print(calculate_self_jaccard_similarity(text_1_0))

{0: 0.14068970833247754}


In [ ]:
text_1_1=df_text1[df_text1["label"]==1].sample(n=1000, random_state=7)
print(calculate_self_jaccard_similarity(text_1_0))

{0: 0.14068970833247754}


In [ ]:
from transformers import BertTokenizer
from transformers import BertForSequenceClassification
bert_tokenizer = BertTokenizer.from_pretrained('/content/drive/MyDrive/planB/bert/BERT_baseline_723_6')
bert_model = BertForSequenceClassification.from_pretrained('/content/drive/MyDrive/planB/bert/BERT_baseline_723_6', num_labels=2)

In [ ]:
import torch
from torch.utils.data import Dataset, DataLoader
class TextDataset(Dataset):
    def __init__(self, dataframe, tokenizer, max_len):
        self.tokenizer = tokenizer
        self.data = dataframe
        self.text = dataframe.text
        self.targets = dataframe.label
        self.max_len = max_len

    def __len__(self):
        return len(self.text)

    def __getitem__(self, index):
        text = str(self.text[index])
        text = " ".join(text.split())

        inputs = self.tokenizer.encode_plus(
            text,
            None,
            add_special_tokens=True,
            max_length=self.max_len,
            padding='max_length',
            truncation=True,
            return_attention_mask=True,
            return_tensors="pt",
        )

        return {
            'input_ids': inputs['input_ids'].flatten(),
            'attention_mask': inputs['attention_mask'].flatten(),
            'labels': torch.tensor(self.targets[index], dtype=torch.long)
        }


In [ ]:
MAX_LENGTH = 320
BATCH_SIZE = 16
test_dataset = TextDataset(dataframe=df_text1, tokenizer=bert_tokenizer, max_len=MAX_LENGTH)
test_loader = DataLoader(test_dataset, batch_size=BATCH_SIZE, shuffle=False)

In [ ]:
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix
import numpy as np

def evaluate(model, dataloader):
    model.eval()  # Put the model in evaluation mode

    predictions, true_labels = [], []

    with torch.no_grad():
        for batch in dataloader:
            input_ids = batch['input_ids'].to(device)
            attention_mask = batch['attention_mask'].to(device)
            labels = batch['labels'].to(device)

            outputs = model(input_ids, attention_mask=attention_mask, labels=labels)

            logits = outputs.logits.detach().cpu().numpy()
            label_ids = labels.to('cpu').numpy()

            predictions.extend(np.argmax(logits, axis=1).flatten())
            true_labels.extend(label_ids.flatten())

    avg_accuracy = accuracy_score(true_labels, predictions)
    print(f'Validation Accuracy: {avg_accuracy}')

    # Detailed classification report
    print("\nClassification Report:\n", classification_report(true_labels, predictions, target_names=['Class0', 'Class1']))

    # Confusion Matrix
    print("Confusion Matrix:\n", confusion_matrix(true_labels, predictions))

In [ ]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
bert_model.to(device)
evaluate(bert_model, test_loader)

Validation Accuracy: 0.9343

Classification Report:
               precision    recall  f1-score   support

      Class0       0.89      1.00      0.94     10000
      Class1       0.99      0.87      0.93     10000

    accuracy                           0.93     20000
   macro avg       0.94      0.93      0.93     20000
weighted avg       0.94      0.93      0.93     20000

Confusion Matrix:
 [[9954   46]
 [1268 8732]]


In [ ]:
from transformers import BartTokenizer, BartForConditionalGeneration
tokenizer = BartTokenizer.from_pretrained("/content/drive/MyDrive/model_722_bart_epoch_4")
model = BartForConditionalGeneration.from_pretrained("/content/drive/MyDrive/model_722_bart_epoch_4")

In [ ]:
import pandas as pd
import torch
device = 'cuda' if torch.cuda.is_available() else 'cpu'
model.eval()
model.to(device)

BartForConditionalGeneration(
  (model): BartModel(
    (shared): Embedding(50265, 768, padding_idx=1)
    (encoder): BartEncoder(
      (embed_tokens): BartScaledWordEmbedding(50265, 768, padding_idx=1)
      (embed_positions): BartLearnedPositionalEmbedding(1026, 768)
      (layers): ModuleList(
        (0-5): 6 x BartEncoderLayer(
          (self_attn): BartSdpaAttention(
            (k_proj): Linear(in_features=768, out_features=768, bias=True)
            (v_proj): Linear(in_features=768, out_features=768, bias=True)
            (q_proj): Linear(in_features=768, out_features=768, bias=True)
            (out_proj): Linear(in_features=768, out_features=768, bias=True)
          )
          (self_attn_layer_norm): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
          (activation_fn): GELUActivation()
          (fc1): Linear(in_features=768, out_features=3072, bias=True)
          (fc2): Linear(in_features=3072, out_features=768, bias=True)
          (final_layer_norm): Laye

In [ ]:
import torch
import pandas as pd
import random
import numpy as np

def set_seeds(seed):
    torch.manual_seed(seed)
    random.seed(seed)
    np.random.seed(seed)

def generate_text_with_beams(prompt, num_texts=5, temperature=1.0, top_k=50, top_p=0.95, min_length=50, max_length=320, num_beams=5, no_repeat_ngram_size=2, early_stopping=True, seed=42, repetition_penalty=1.5):
    if seed is not None:
        set_seeds(seed)
    model.eval()
    generated_texts = []
    for _ in range(num_texts):
        input_ids = tokenizer.encode(prompt, return_tensors='pt').to(model.device)

        outputs = model.generate(
            input_ids,
            min_length=min_length,
            max_length=max_length,
            num_beams=num_beams,
            num_return_sequences=1,  # Generate one sequence at a time to ensure diversity
            temperature=temperature,
            top_k=top_k,
            top_p=top_p,
            no_repeat_ngram_size=no_repeat_ngram_size,
            repetition_penalty=repetition_penalty,
            do_sample=True,
            pad_token_id=tokenizer.eos_token_id,
            early_stopping=early_stopping
        )

        text = tokenizer.decode(outputs[0], skip_special_tokens=True)
        generated_texts.append(text)

    return generated_texts

def generate_text(top_p, top_k, temperature, num_beams=5, num_texts=20, seed=42, repetition_penalty=1.2):
    prompts = ["Write a World News", "Write a Sports News", "Write a Business News", "Write a Science/Technology News"]
    all_prompts = []
    all_texts = []

    for prompt in prompts:
        generated_texts_for_prompt = []
        num_iterations = num_texts // num_beams

        for i in range(num_iterations):
            set_seeds(seed + i)
            generated_texts = generate_text_with_beams(
                prompt=prompt,
                num_texts=num_beams,
                temperature=temperature,
                top_k=top_k,
                top_p=top_p,
                min_length=50,
                max_length=200,  # Adjust max_length as needed
                num_beams=num_beams,
                no_repeat_ngram_size=2,
                early_stopping=True,  # Enable early stopping
                seed=seed + i,  # Adjust seed for each batch to maintain diversity while being reproducible
                repetition_penalty=repetition_penalty  # Adding repetition penalty
            )
            generated_texts_for_prompt.extend(generated_texts)

        # Ensure uniqueness and limit to the desired number of texts
        unique_texts = list(set(generated_texts_for_prompt))[:num_texts]
        all_prompts.extend([prompt] * len(unique_texts))
        all_texts.extend(unique_texts)

    return pd.DataFrame({'prompt': all_prompts, 'text': all_texts})

In [ ]:
import torch
from torch.utils.data import Dataset, DataLoader
class TextDataset(Dataset):
    def __init__(self, dataframe, tokenizer, max_len=320):
        self.tokenizer = tokenizer
        self.data = dataframe
        self.text = dataframe.text
        self.targets = dataframe.label
        self.max_len = max_len

    def __len__(self):
        return len(self.text)

    def __getitem__(self, index):
        text = str(self.text[index])
        text = " ".join(text.split())

        inputs = self.tokenizer.encode_plus(
            text,
            None,
            add_special_tokens=True,
            max_length=self.max_len,
            padding='max_length',
            truncation=True,
            return_attention_mask=True,
            return_tensors="pt",
        )

        return {
            'input_ids': inputs['input_ids'].flatten(),
            'attention_mask': inputs['attention_mask'].flatten(),
            'labels': torch.tensor(self.targets[index], dtype=torch.long)
        }


In [ ]:
import pandas as pd
from sklearn.metrics import jaccard_score
from itertools import combinations
import numpy as np

# Function to calculate Jaccard Similarity between two texts
def jaccard_similarity(text1, text2):
    set1 = set(text1.split())
    set2 = set(text2.split())
    intersection = set1.intersection(set2)
    union = set1.union(set2)
    return len(intersection) / len(union) if len(union) > 0 else 0

# Function to calculate mean Jaccard similarity within each class
def calculate_self_jaccard_similarity(df):
    results = {}
    for label, group in df.groupby('label'):
        jaccard_similarities = [jaccard_similarity(text1, text2) for text1, text2 in combinations(group['text'], 2)]
        results[label] = np.mean(jaccard_similarities) if jaccard_similarities else 0
    return results

In [ ]:
from transformers import BertTokenizer, BertForSequenceClassification
import torch
from sklearn.metrics import accuracy_score
import numpy as np
from collections import defaultdict

# Load the tokenizer and model
bert_tokenizer = BertTokenizer.from_pretrained('/content/drive/MyDrive/planB/bert/BERT611_half_2')
bert_model = BertForSequenceClassification.from_pretrained('/content/drive/MyDrive/planB/bert/BERT611_half_2', num_labels=4)


bert_model_test = BertForSequenceClassification.from_pretrained('/content/drive/MyDrive/planB/bert/BERT613_half_2', num_labels=4)
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
bert_model.to(device)
bert_model_test.to(device)

def evaluate(model, dataloader):
    model.eval()

    predictions, true_labels = [], []

    with torch.no_grad():
        for batch in dataloader:
            input_ids = batch['input_ids'].to(device)
            attention_mask = batch['attention_mask'].to(device)
            labels = batch['labels'].to(device)

            outputs = model(input_ids, attention_mask=attention_mask, labels=labels)

            logits = outputs.logits.detach().cpu().numpy()
            label_ids = labels.to('cpu').numpy()

            predictions.extend(np.argmax(logits, axis=1).flatten())
            true_labels.extend(label_ids.flatten())

    avg_accuracy = accuracy_score(true_labels, predictions)

    # Calculate accuracy for each class
    class_accuracies = defaultdict(list)
    for true, pred in zip(true_labels, predictions):
        class_accuracies[true].append(pred == true)

    class_accuracy_results = {label: np.mean(acc) for label, acc in class_accuracies.items()}

    return avg_accuracy, class_accuracy_results

In [ ]:
!pip install sacrebleu

import pandas as pd
import numpy as np
import sacrebleu

def calculate_self_bleu(texts):
    scores = []
    for i, target in enumerate(texts):
        references = texts[:i] + texts[i+1:]
        bleu_score = sacrebleu.corpus_bleu([target], [references]).score
        scores.append(bleu_score)
    return np.mean(scores)

def calculate_self_bleu_per_class(df):
    class_bleu_scores = {}
    for label, group in df.groupby('label'):
        texts = group['text'].tolist()
        self_bleu_score = calculate_self_bleu(texts)
        class_bleu_scores[label] = self_bleu_score
    return class_bleu_scores

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 106.7/106.7 kB 4.2 MB/s eta 0:00:00


In [ ]:
label_map2 = {"Write a World News":0, "Write a Sports News":1, "Write a Business News":2, "Write a Science/Technology News":3}
# label_map2 = {"World News":0, "Sport News":1, "Business News":2, "Science/Technology News":3}

In [ ]:
df_text = generate_text(top_p=0.95, top_k=50, temperature=1.3, num_beams=4 , num_texts=10, repetition_penalty=1,seed=611)

KeyboardInterrupt: 

In [ ]:
df_text.text.to_list()

['Iraqi troops in Iraq Kill At Least 39 Suspects Reuters Reuters - Iraqi troops killed at least 39 suspected militants in a raid on a military base in northern Iraq, officials said on Friday, adding that the militants had been killed and 39 wounded.',
 'Pakistan suspends military action in Kashmir  AP AP - Pakistan has suspended military operations in the Kashmir valley as part of an effort to restore peace to the region, a senior Pakistani military official said on Saturday. The military said the decision was made in response to a report from the UN Security Council.',
 'Pakistani Journalists Killed in Afghanistan Reuters - Pakistani Journalists were killed in a helicopter crash near the border with Afghanistan on Saturday, killing at least three people and wounding dozens, police said. The military said the bodies were recovered from the wreckage of the helicopter.',
 'U.S. troops in Iraq, Afghanistan Reuters - The US military said on Friday it had made an initial military deployment

In [ ]:
'''
  dataset = TextDataset(dataframe=df_BART, tokenizer=bert_tokenizer, max_len=320)
  loader = DataLoader(dataset, batch_size=8, shuffle=False)
  bert_acc = evaluate(bert_model, loader)
  bert_acc_ag = evaluate(bert_model_ag, loader)
  js = calculate_self_jaccard_similarity(df_text)
  sb = calculate_self_bleu_per_class(df_text)
  print(f"top_k: {k}")
  print(f"bert accuracy: {bert_acc}")
  print(f"Jaccard similarities: {js}")
  print(f"Self-BLEU: {sb}")
  '''
  print()

top_k: 53
bert accuracy: (0.96, {1: 0.9900990099009901, 3: 0.9411764705882353, 0: 0.9578947368421052, 2: 0.9509803921568627})
Jaccard similarities: {0: 0.1546937782671968, 1: 0.12329236104122708, 2: 0.1357759885645806, 3: 0.10681470354254556}
Self-BLEU: {0: 5.539936406049439, 1: 2.3601559338398683, 2: 6.301246408694692, 3: 2.9587081236278174}


In [ ]:
for k in [47,50,53,55]:
  df_text = generate_text(top_p=0.95, top_k=k, temperature=1.3, num_beams=4 , num_texts=100, repetition_penalty=1.7,seed=611)
  df_text['label'] = df_text['prompt'].map(label_map2)
  dataset = TextDataset(dataframe=df_text, tokenizer=bert_tokenizer, max_len=320)
  loader = DataLoader(dataset, batch_size=16, shuffle=False)
  bert_acc = evaluate(bert_model, loader)
  bert_acc_test = evaluate(bert_model_test, loader)
  js = calculate_self_jaccard_similarity(df_text)
  sb = calculate_self_bleu_per_class(df_text)
  print(f"top_k: {k}")
  print(f"bert accuracy: {bert_acc}")
  print(f"bert accuracy test: {bert_acc_test}")
  print(f"Jaccard similarities: {js}")
  print(f"Self-BLEU: {sb}")

top_k: 47
bert accuracy: (0.985, {0: 0.98, 1: 1.0, 2: 0.98, 3: 0.98})
bert accuracy test: (0.9775, {0: 0.97, 1: 1.0, 2: 0.98, 3: 0.96})
Jaccard similarities: {0: 0.14067809893271552, 1: 0.14401190533520303, 2: 0.16369632454867825, 3: 0.17209882496834944}
Self-BLEU: {0: 4.131965716311886, 1: 4.623346079613982, 2: 11.378262467280184, 3: 6.129949725486852}
top_k: 50
bert accuracy: (0.975, {0: 0.97, 1: 0.99, 2: 0.97, 3: 0.97})
bert accuracy test: (0.975, {0: 0.97, 1: 0.99, 2: 0.98, 3: 0.96})
Jaccard similarities: {0: 0.13497639999035846, 1: 0.1416439360858013, 2: 0.1742633315104432, 3: 0.1686567306459168}
Self-BLEU: {0: 2.3459336058584994, 1: 4.2850660619368375, 2: 12.244161065330607, 3: 5.579620159495602}
top_k: 53
bert accuracy: (0.9825, {0: 0.99, 1: 1.0, 2: 0.98, 3: 0.96})
bert accuracy test: (0.985, {0: 0.99, 1: 1.0, 2: 0.99, 3: 0.96})
Jaccard similarities: {0: 0.13677639351999893, 1: 0.14227985594404347, 2: 0.16424890461723762, 3: 0.16362226705529348}
Self-BLEU: {0: 3.514322150376501,

In [ ]:
for k in [47,50,53,55]:
  df_text = generate_text(top_p=0.95, top_k=k, temperature=1.3, num_beams=4 , num_texts=100, repetition_penalty=1.7,seed=611)
  df_text['label'] = df_text['prompt'].map(label_map2)
  dataset = TextDataset(dataframe=df_text, tokenizer=bert_tokenizer, max_len=320)
  loader = DataLoader(dataset, batch_size=16, shuffle=False)
  bert_acc = evaluate(bert_model, loader)
  bert_acc_test = evaluate(bert_model_test, loader)
  js = calculate_self_jaccard_similarity(df_text)
  sb = calculate_self_bleu_per_class(df_text)
  print(f"top_k: {k}")
  print(f"bert accuracy: {bert_acc}")
  print(f"bert accuracy test: {bert_acc_test}")
  print(f"Jaccard similarities: {js}")
  print(f"Self-BLEU: {sb}")

top_k: 47
bert accuracy: (0.9775, {0: 0.98, 1: 0.98, 2: 0.99, 3: 0.96})
bert accuracy test: (0.9775, {0: 0.98, 1: 0.98, 2: 0.99, 3: 0.96})
Jaccard similarities: {0: 0.14767446853242291, 1: 0.14051381836245597, 2: 0.15316019859630012, 3: 0.15512926154121254}
Self-BLEU: {0: 2.4658698937674193, 1: 5.543504553618494, 2: 4.847317171514992, 3: 4.349677943518833}
top_k: 50
bert accuracy: (0.9975, {0: 0.99, 1: 1.0, 2: 1.0, 3: 1.0})
bert accuracy test: (0.9875, {0: 0.99, 1: 0.99, 2: 0.97, 3: 1.0})
Jaccard similarities: {0: 0.1521805463826348, 1: 0.1385159116259844, 2: 0.15683318223982703, 3: 0.1584736139743345}
Self-BLEU: {0: 8.76729684820211, 1: 3.8357909175220084, 2: 5.928027277612154, 3: 5.829476614814385}
top_k: 53
bert accuracy: (0.9875, {0: 1.0, 1: 1.0, 2: 1.0, 3: 0.95})
bert accuracy test: (0.985, {0: 1.0, 1: 0.99, 2: 1.0, 3: 0.95})
Jaccard similarities: {0: 0.15440322092434722, 1: 0.1503552948935853, 2: 0.15162540572529487, 3: 0.15903208018695053}
Self-BLEU: {0: 6.193909863424978, 1: 6.

In [ ]:
for t in [1.2,1.4,1.5,1.6]:
  df_text = generate_text(top_p=0.95, top_k=50, temperature=t, num_beams=4 , num_texts=100, repetition_penalty=1,seed=611)
  df_text['label'] = df_text['prompt'].map(label_map2)
  dataset = TextDataset(dataframe=df_text, tokenizer=bert_tokenizer, max_len=320)
  loader = DataLoader(dataset, batch_size=16, shuffle=False)
  bert_acc = evaluate(bert_model, loader)
  bert_acc_test = evaluate(bert_model_test, loader)
  js = calculate_self_jaccard_similarity(df_text)
  sb = calculate_self_bleu_per_class(df_text)
  print(f"temp: {t}")
  print(f"bert accuracy: {bert_acc}")
  print(f"bert accuracy test: {bert_acc_test}")
  print(f"Jaccard similarities: {js}")
  print(f"Self-BLEU: {sb}")

temp: 1.2
bert accuracy: (0.9775, {0: 0.98, 1: 1.0, 2: 0.99, 3: 0.94})
bert accuracy test: (0.975, {0: 0.99, 1: 1.0, 2: 0.99, 3: 0.92})
Jaccard similarities: {0: 0.1535967301500334, 1: 0.14290038251828768, 2: 0.1536245452585003, 3: 0.15541633547501504}
Self-BLEU: {0: 5.4452125559338755, 1: 5.619466506347688, 2: 7.604275243694099, 3: 8.456351037922772}
temp: 1.4
bert accuracy: (0.9825, {0: 0.98, 1: 1.0, 2: 0.97, 3: 0.98})
bert accuracy test: (0.975, {0: 0.98, 1: 1.0, 2: 0.95, 3: 0.97})
Jaccard similarities: {0: 0.13670405511274863, 1: 0.12527183089103627, 2: 0.13330846517434275, 3: 0.1469298341837145}
Self-BLEU: {0: 3.7210924983341114, 1: 5.74634037433728, 2: 2.924648080452917, 3: 8.270848019494315}
temp: 1.5
bert accuracy: (0.995, {0: 0.99, 1: 1.0, 2: 1.0, 3: 0.99})
bert accuracy test: (0.99, {0: 1.0, 1: 1.0, 2: 0.99, 3: 0.97})
Jaccard similarities: {0: 0.14728521874484718, 1: 0.13133402208072784, 2: 0.14607243283313287, 3: 0.13344427170111628}
Self-BLEU: {0: 4.49005701488763, 1: 5.385

In [ ]:
for t in [1.2,1.4,1.5,1.6]:
  df_text = generate_text(top_p=0.95, top_k=50, temperature=t, num_beams=4 , num_texts=100, repetition_penalty=1.7,seed=611)
  df_text['label'] = df_text['prompt'].map(label_map2)
  dataset = TextDataset(dataframe=df_text, tokenizer=bert_tokenizer, max_len=320)
  loader = DataLoader(dataset, batch_size=16, shuffle=False)
  bert_acc = evaluate(bert_model, loader)
  bert_acc_test = evaluate(bert_model_test, loader)
  js = calculate_self_jaccard_similarity(df_text)
  sb = calculate_self_bleu_per_class(df_text)
  print(f"temp: {t}")
  print(f"bert accuracy: {bert_acc}")
  print(f"bert accuracy test: {bert_acc_test}")
  print(f"Jaccard similarities: {js}")
  print(f"Self-BLEU: {sb}")

temp: 1.2
bert accuracy: (0.99, {0: 0.99, 1: 1.0, 2: 1.0, 3: 0.97})
bert accuracy test: (0.9925, {0: 1.0, 1: 1.0, 2: 1.0, 3: 0.97})
Jaccard similarities: {0: 0.16692469495680134, 1: 0.14255364179542554, 2: 0.16962849725828444, 3: 0.16892443053992204}
Self-BLEU: {0: 7.694385121300005, 1: 4.101871484633133, 2: 10.240818888516358, 3: 4.302335119658982}
temp: 1.4
bert accuracy: (0.98, {0: 0.99, 1: 1.0, 2: 1.0, 3: 0.93})
bert accuracy test: (0.9775, {0: 0.99, 1: 1.0, 2: 1.0, 3: 0.92})
Jaccard similarities: {0: 0.1455908403254702, 1: 0.13370877565836686, 2: 0.1415974253889051, 3: 0.15460546763326138}
Self-BLEU: {0: 3.9533197662363206, 1: 4.298911077510723, 2: 3.307725955786069, 3: 5.731214337494558}


KeyboardInterrupt: 

In [ ]:
for t in [1.2,1.4,1.5,1.6]:
  df_text = generate_text(top_p=0.95, top_k=55, temperature=t, num_beams=4 , num_texts=100, repetition_penalty=1.7,seed=611)
  df_text['label'] = df_text['prompt'].map(label_map2)
  dataset = TextDataset(dataframe=df_text, tokenizer=bert_tokenizer, max_len=320)
  loader = DataLoader(dataset, batch_size=16, shuffle=False)
  bert_acc = evaluate(bert_model, loader)
  bert_acc_test = evaluate(bert_model_test, loader)
  js = calculate_self_jaccard_similarity(df_text)
  sb = calculate_self_bleu_per_class(df_text)
  print(f"temp: {t}")
  print(f"bert accuracy: {bert_acc}")
  print(f"bert accuracy test: {bert_acc_test}")
  print(f"Jaccard similarities: {js}")
  print(f"Self-BLEU: {sb}")

temp: 1.2
bert accuracy: (0.98, {0: 0.96, 1: 1.0, 2: 0.99, 3: 0.97})
bert accuracy test: (0.9725, {0: 0.98, 1: 0.99, 2: 0.95, 3: 0.97})
Jaccard similarities: {0: 0.15641776862493517, 1: 0.13762651268851406, 2: 0.14942832080177632, 3: 0.17096600377332957}
Self-BLEU: {0: 3.6973724694662113, 1: 5.510478748692128, 2: 5.282894727860917, 3: 6.148517202395925}
temp: 1.4
bert accuracy: (0.99, {0: 0.99, 1: 1.0, 2: 1.0, 3: 0.97})
bert accuracy test: (0.9825, {0: 0.99, 1: 1.0, 2: 0.99, 3: 0.95})
Jaccard similarities: {0: 0.1549490423853397, 1: 0.13609098478029463, 2: 0.13867162824248058, 3: 0.1565163668826492}
Self-BLEU: {0: 3.700804454163212, 1: 4.848572425849049, 2: 8.176757883651408, 3: 4.162351425753526}
temp: 1.5
bert accuracy: (0.9825, {0: 0.99, 1: 1.0, 2: 0.97, 3: 0.97})
bert accuracy test: (0.9775, {0: 0.99, 1: 1.0, 2: 0.96, 3: 0.96})
Jaccard similarities: {0: 0.13763309906493776, 1: 0.13379551786827815, 2: 0.1419002132679115, 3: 0.14626787246502562}
Self-BLEU: {0: 3.541654337458682, 1: 3

In [ ]:
df_text0 = generate_text(top_p=0.95, top_k=50, temperature=1.3, num_beams=4 , num_texts=600, repetition_penalty=1.7,seed=611)
df_text0['label'] = df_text0['prompt'].map(label_map2)
df_text0.to_csv("/content/drive/MyDrive/planB/df_generated_bartall_1.csv")

In [ ]:
df_text0 = generate_text(top_p=0.95, top_k=55, temperature=1.3, num_beams=4 , num_texts=600, repetition_penalty=1.7,seed=611)
df_text0['label'] = df_text0['prompt'].map(label_map2)
df_text0.to_csv("/content/drive/MyDrive/planB/df_generated_bartall_0.csv")

In [ ]:
df_text0 = generate_text(top_p=0.95, top_k=55, temperature=1.6, num_beams=4 , num_texts=600, repetition_penalty=1,seed=611)
df_text0['label'] = df_text0['prompt'].map(label_map2)
df_text0.to_csv("/content/drive/MyDrive/planB/df_generated_bartall_2.csv")

In [ ]:
## all 2
dataset = TextDataset(dataframe=df_text0, tokenizer=bert_tokenizer, max_len=320)
loader = DataLoader(dataset, batch_size=16, shuffle=False)
bert_acc = evaluate(bert_model, loader)
bert_acc_test = evaluate(bert_model_test, loader)
js = calculate_self_jaccard_similarity(df_text0)
sb = calculate_self_bleu_per_class(df_text0)

print(f"bert accuracy: {bert_acc}")
print(f"bert accuracy test: {bert_acc_test}")
print(f"Jaccard similarities: {js}")
print(f"Self-BLEU: {sb}")

bert accuracy: (0.9775, {0: 0.9883333333333333, 1: 0.995, 2: 0.9783333333333334, 3: 0.9483333333333334})
bert accuracy test: (0.97375, {0: 0.99, 1: 0.9916666666666667, 2: 0.9716666666666667, 3: 0.9416666666666667})
Jaccard similarities: {0: 0.12852174870001049, 1: 0.12118574728904863, 2: 0.12630426061016808, 3: 0.12701263847992897}
Self-BLEU: {0: 1.6222599773364461, 1: 3.3259993130776806, 2: 6.913054882490829, 3: 3.602974141980892}


In [ ]:
## all 1
dataset = TextDataset(dataframe=df_text0, tokenizer=bert_tokenizer, max_len=320)
loader = DataLoader(dataset, batch_size=16, shuffle=False)
bert_acc = evaluate(bert_model, loader)
bert_acc_test = evaluate(bert_model_test, loader)
js = calculate_self_jaccard_similarity(df_text0)
sb = calculate_self_bleu_per_class(df_text0)

print(f"bert accuracy: {bert_acc}")
print(f"bert accuracy test: {bert_acc_test}")
print(f"Jaccard similarities: {js}")
print(f"Self-BLEU: {sb}")

bert accuracy: (0.9866666666666667, {0: 0.9866666666666667, 1: 0.9966666666666667, 2: 0.9916666666666667, 3: 0.9716666666666667})
bert accuracy test: (0.9845833333333334, {0: 0.9933333333333333, 1: 0.9933333333333333, 2: 0.9816666666666667, 3: 0.97})
Jaccard similarities: {0: 0.15792642113542213, 1: 0.1430546126739633, 2: 0.1541496428375523, 3: 0.1617372274052592}
Self-BLEU: {0: 6.640209666101523, 1: 5.3373389182610635, 2: 6.38432482999989, 3: 4.705580714887631}


In [ ]:
## all 0
dataset = TextDataset(dataframe=df_text0, tokenizer=bert_tokenizer, max_len=320)
loader = DataLoader(dataset, batch_size=16, shuffle=False)
bert_acc = evaluate(bert_model, loader)
bert_acc_test = evaluate(bert_model_test, loader)
js = calculate_self_jaccard_similarity(df_text0)
sb = calculate_self_bleu_per_class(df_text0)

print(f"bert accuracy: {bert_acc}")
print(f"bert accuracy test: {bert_acc_test}")
print(f"Jaccard similarities: {js}")
print(f"Self-BLEU: {sb}")

bert accuracy: (0.9845833333333334, {0: 0.99, 1: 0.9966666666666667, 2: 0.995, 3: 0.9566666666666667})
bert accuracy test: (0.9829166666666667, {0: 0.995, 1: 0.9866666666666667, 2: 0.985, 3: 0.965})
Jaccard similarities: {0: 0.15050207066069718, 1: 0.140960365250401, 2: 0.15147204123493685, 3: 0.1570379000011103}
Self-BLEU: {0: 5.6813036315698335, 1: 4.333808496890731, 2: 11.426461986735081, 3: 6.113334556653601}


In [ ]:
df_text0 = generate_text(top_p=0.95, top_k=55, temperature=1.3, num_beams=4 , num_texts=600, repetition_penalty=1.7,seed=611)
df_text0['label'] = df_text0['prompt'].map(label_map2)
df_text0.to_csv("/content/drive/MyDrive/planB/df_generated_bart300_2.csv")

In [ ]:
df_text0 = generate_text(top_p=0.95, top_k=55, temperature=1.3, num_beams=4 , num_texts=600, repetition_penalty=1.9,seed=611)
df_text0['label'] = df_text0['prompt'].map(label_map2)
df_text0.to_csv("/content/drive/MyDrive/planB/df_generated_bart300_3.csv")

In [ ]:
df_text0=pd.read_csv("/content/drive/MyDrive/planB/df_generated_bart300_0.csv")

In [ ]:
dataset = TextDataset(dataframe=df_text0, tokenizer=bert_tokenizer, max_len=320)
loader = DataLoader(dataset, batch_size=16, shuffle=False)
bert_acc = evaluate(bert_model, loader)
bert_acc_test = evaluate(bert_model_test, loader)
js = calculate_self_jaccard_similarity(df_text0)
sb = calculate_self_bleu_per_class(df_text0)

print(f"bert accuracy: {bert_acc}")
print(f"bert accuracy test: {bert_acc_test}")
print(f"Jaccard similarities: {js}")
print(f"Self-BLEU: {sb}")

bert accuracy: (0.9816666666666667, {0: 0.99, 1: 0.9916666666666667, 2: 0.98, 3: 0.965})
bert accuracy test: (0.9770833333333333, {0: 0.99, 1: 0.9883333333333333, 2: 0.9633333333333334, 3: 0.9666666666666667})
Jaccard similarities: {0: 0.141587262913951, 1: 0.12340321118667312, 2: 0.1515325593266859, 3: 0.17569754554733163}
Self-BLEU: {0: 2.5195661219666476, 1: 6.129392816229647, 2: 9.507321405085314, 3: 8.557186187687437}


In [ ]:
dataset = TextDataset(dataframe=df_text0, tokenizer=bert_tokenizer, max_len=320)
loader = DataLoader(dataset, batch_size=16, shuffle=False)
bert_acc = evaluate(bert_model, loader)
bert_acc_test = evaluate(bert_model_test, loader)
js = calculate_self_jaccard_similarity(df_text0)
sb = calculate_self_bleu_per_class(df_text0)

print(f"bert accuracy: {bert_acc}")
print(f"bert accuracy test: {bert_acc_test}")
print(f"Jaccard similarities: {js}")
print(f"Self-BLEU: {sb}")

bert accuracy: (0.9841666666666666, {0: 0.9966666666666667, 1: 0.9916666666666667, 2: 0.9833333333333333, 3: 0.965})
bert accuracy test: (0.9779166666666667, {0: 0.995, 1: 0.9883333333333333, 2: 0.9683333333333334, 3: 0.96})
Jaccard similarities: {0: 0.13586357238209182, 1: 0.11845386333304575, 2: 0.14952601976826219, 3: 0.1557771000804527}
Self-BLEU: {0: 4.669816859370167, 1: 3.4167383927607844, 2: 4.234192052148225, 3: 2.695258355083081}


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
import nbformat

# Path to your notebook
path = '/content/drive/MyDrive/Colab Notebooks/BART_generate.ipynb'

# Load notebook
with open(path) as f:
    nb = nbformat.read(f, as_version=4)

# Clean metadata
if 'widgets' in nb['metadata']:
    del nb['metadata']['widgets']
    print("Removed 'widgets' metadata.")
else:
    print("No 'widgets' metadata found.")

# Save the cleaned notebook (overwrite original)
with open(path, 'w') as f:
    nbformat.write(nb, f)

print("Notebook cleaned and saved.")


No 'widgets' metadata found.
Notebook cleaned and saved.
